In [ ]:
# based on commit 8d5f7 "aaaaaaa" something something
# anything since then needs updating manually in the database

In [5]:
import sqlite3
import base64
from PIL import Image
import io
import os
from datetime import datetime

def compress_image(image_path, max_size=512 * 1024):
    with Image.open(image_path) as img:
        # Reduce image quality until it is below the max_size
        quality = 90
        
        while True:
            buffer = io.BytesIO()
            img.save(buffer, format="JPEG", quality=quality)
            size = buffer.tell()
            
            if size <= max_size:
                break
            
            if quality > 20:
                quality -= 5
            else:
                width, height = img.size
                if width > 1 and height > 1:
                    img = img.resize((width // 2, height // 2), Image.Resampling.LANCZOS)
                    quality = 90
                else:
                    break
        print("quality:", quality)
        print("width, height:", img.size)
        buffer.seek(0)
        return buffer

def image_to_base64(image_path):
    compressed_image = compress_image(image_path)
    base64_bytes = base64.b64encode(compressed_image.read()).decode('utf-8')
    return base64_bytes

def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

def insert_image_if_not_exists(db_path, image_path):
    # Get image dimensions
    width, height = get_image_dimensions(image_path)
    
    # Convert the image to Base64
    base64_string = image_to_base64(image_path)
    
    # Get the filename as the description without the extension
    description = os.path.splitext(os.path.basename(image_path))[0]

    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Check if an image with the same base64_bytes already exists
    cursor.execute('''
        SELECT id FROM images WHERE base64_bytes = ?
    ''', (base64_string,))

    existing_image = cursor.fetchone()

    if existing_image:
        # If an image already exists, return its ID
        image_id = existing_image[0]
        print("Image already exists with ID:", image_id)
    else:
        # Insert a new row into the images table
        cursor.execute('''
            INSERT INTO images (width, height, description, base64_bytes, created_at)
            VALUES (?, ?, ?, ?, CURRENT_TIMESTAMP)
        ''', (width, height, description, base64_string))

        # Get the ID of the newly inserted row
        image_id = cursor.lastrowid
        print("New image inserted with ID:", image_id)

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    return image_id

def insert_update_if_not_exists(db_path, created_at, description, cave_id):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Check if an update with the same description, cave_id, and created_at already exists
    cursor.execute('''
        SELECT id FROM updates WHERE description = ? AND cave_id = ? AND created_at = ?
    ''', (description, cave_id, created_at))

    existing_update = cursor.fetchone()

    if existing_update:
        # If an update already exists, return its ID
        update_id = existing_update[0]
        print("Update already exists with ID:", update_id)
    else:
        # Insert a new row into the updates table
        cursor.execute('''
            INSERT INTO updates (created_at, description, cave_id)
            VALUES (?, ?, ?)
        ''', (created_at, description, cave_id))

        # Get the ID of the newly inserted row
        update_id = cursor.lastrowid
        print("New update inserted with ID:", update_id)

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    return update_id

def get_cave_id_by_name(db_path, cave_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Query to get the ID of the cave with the given name
    cursor.execute('''
        SELECT id FROM caves WHERE name = ?
    ''', (cave_name,))

    # Fetch the result
    cave_id = cursor.fetchone()

    # Close the connection
    conn.close()

    # Check if the cave was found and return the ID, otherwise return None
    if cave_id:
        return cave_id[0]
    else:
        return None

def insert_cave_if_not_exists(db_path, cave_name, latitude, longitude, elevation):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Check if a cave with the same name already exists
    cursor.execute('''
        SELECT id FROM caves WHERE name = ?
    ''', (cave_name,))

    existing_cave = cursor.fetchone()

    if existing_cave:
        # If a cave already exists, return its ID
        cave_id = existing_cave[0]
        print("Cave already exists with ID:", cave_id)
    else:
        # Insert a new row into the caves table
        cursor.execute('''
            INSERT INTO caves (name, latitude, longitude, elevation)
            VALUES (?, ?, ?, ?)
        ''', (cave_name, latitude, longitude, elevation))

        # Get the ID of the newly inserted row
        cave_id = cursor.lastrowid
        print("New cave inserted with ID:", cave_id)

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    return cave_id

def get_datetime_for_year(year_str):
    # Construct a datetime object for July 1st of the given year
    date_str = f"{year_str}-07-01 00:00:00"
    date_obj = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    
    # Format the datetime object as a string suitable for SQLite insertion
    formatted_date = date_obj.strftime("%Y-%m-%d %H:%M:%S")
    
    return formatted_date

def insert_update_image_if_not_exists(db_path, update_id, image_id):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Check if the (update_id, image_id) combination already exists
    cursor.execute('''
        SELECT 1 FROM update_images WHERE update_id = ? AND image_id = ?
    ''', (update_id, image_id))

    existing_entry = cursor.fetchone()

    if existing_entry:
        # If the entry already exists, print a message
        print(f"Entry already exists for update_id: {update_id} and image_id: {image_id}")
    else:
        # Insert a new row into the UpdateImages table
        cursor.execute('''
            INSERT INTO update_images (update_id, image_id)
            VALUES (?, ?)
        ''', (update_id, image_id))

        # Commit the changes and close the connection
        conn.commit()
        print(f"New entry inserted for update_id: {update_id} and image_id: {image_id}")

    conn.close()

def image_short_list_to_path(images):
    image_paths = [f"./images/{image}" for image in images]
    return image_paths

# Usage
db_path = '../../../surface-atlas/assets/atlas.db'
image_path = './images/A1-2022.jpeg'

# new_id = insert_image_to_db(db_path, image_path)


In [4]:
# Check all caves exist
sections = atlas_text.split("### ")
caves = [section.split("\n")[0] for section in sections]
for cave in caves:
    if len(cave) > 0:
        insert_cave_if_not_exists(db_path, cave, None, None, None)

Cave already exists with ID: 48
Cave already exists with ID: 52
Cave already exists with ID: 49
Cave already exists with ID: 51
Cave already exists with ID: 107
Cave already exists with ID: 108
Cave already exists with ID: 47
Cave already exists with ID: 20
Cave already exists with ID: 80
Cave already exists with ID: 32
Cave already exists with ID: 17
Cave already exists with ID: 78
Cave already exists with ID: 79
Cave already exists with ID: 109
Cave already exists with ID: 110
Cave already exists with ID: 111
Cave already exists with ID: 112
Cave already exists with ID: 113
Cave already exists with ID: 114
Cave already exists with ID: 115
Cave already exists with ID: 116
Cave already exists with ID: 37
Cave already exists with ID: 72
Cave already exists with ID: 73
Cave already exists with ID: 118
Cave already exists with ID: 71
Cave already exists with ID: 120
Cave already exists with ID: 59
Cave already exists with ID: 61
Cave already exists with ID: 121
Cave already exists with ID

In [1]:
def create_updates_and_images(cave_name, year, images, description):
    if (not cave_name or not year or (not images and not description)):
        raise ValueError("Missing an argument!")

    image_paths = image_short_list_to_path(images)
    cave_id = get_cave_id_by_name(db_path, cave_name)
    timestamp = get_datetime_for_year(year)
    image_ids = [insert_image_if_not_exists(db_path, image_path) for image_path in image_paths]
    update_id = insert_update_if_not_exists(db_path, timestamp, description, cave_id)
    [insert_update_image_if_not_exists(db_path, update_id, image_id) for image_id in image_ids]
    

In [9]:
# create all the updates!

create_updates_and_images("A1", "2008", [], '''- Discovered in 2008 [HM2 draft, page 44]
  - No information other than that they were logged and they are "worth a go down and look"
''')
create_updates_and_images("A1", "2022", ["A1-2022.jpeg"], '''- Relogged in 2022
  - Rhys remembers going down one of them once upon a time so likely dead?
  - A1 Has two stainless bolts in it
''')

create_updates_and_images("A2", "2008", [], '''- Discovered in 2008 [HM2 draft, page 44]
  - No information other than that they were logged and they are "worth a go down and look"
''')
create_updates_and_images("A2", "2022", ["A2-2022.jpeg"], '''- Relogged in 2022
  - Rhys remembers going down one of them once upon a time so likely dead?
''')

create_updates_and_images("A3", "2008", [], '''- Discovered in 2008 [HM2 draft, page 44]
  - No information other than that they were logged and they are "worth a go down and look"
''')
create_updates_and_images("A3", "2022", ["A3-2022.jpeg"], '''- Relogged in 2022
  - Rhys remembers going down one of them once upon a time so likely dead?
''')

create_updates_and_images("A4", "2008", [], '''- Discovered in 2008 [HM2 draft, page 44]
  - No information other than that they were logged and they are "worth a go down and look"
''')
create_updates_and_images("A4", "2022", ["A4-2022.jpeg"], '''- Relogged in 2022
  - Rhys remembers going down one of them once upon a time so likely dead?
''')

create_updates_and_images(cave_name="ABBA 4", year="2019", images=[], description="""- Possible cave entrance found by BenR on the east face of planje during N9 trip in 2019
- ABBA 4 is likely to be 46.2676, 13.75168
- Should revisit to confirm""")

create_updates_and_images(cave_name="ABBA 5", year="2019", images=[], description="""- Possible cave entrance found by BenR on the east face of planje during N9 trip in 2019
- ABBA 5 is likely to be 46.26749, 13.75123
- Should revisit to confirm""")

create_updates_and_images(cave_name="Andy's Secret Dig", year="2022", images=["Andy's Dig-2022.jpeg"], description="""- No longer a secret as of 2022
  - Relogged on the GPS
  - Still diggable but pretty sketchy, with tent poles holding up boulders
  - Reached by going along the old Mig path before turning down a gulley
  - Can also be reached from below, walking up the gulley
  - From the path down the Gulley, climb up towards an obvious alcove
  - Dig is in the back left of the alcove, and down a 5m free climb""")

create_updates_and_images(cave_name="B5", year="2022", images=["B5-2022.jpeg", "B5 Close Up-2022.jpeg"], description="""- Spotted in 2022 from red paint marking
  - No known information
  - Gravel filled and would require a lot of work
  - Very mossy, not promising""")

create_updates_and_images(cave_name="B10", year="1995", images=[], description="""First discovered during the blowing holes recce of 1995 [HM3, page 61, page 104]""")
create_updates_and_images(cave_name="B10", year="2010", images=["B10-2010.jpg"], description="""[HM3, page 61, page 104]""")
create_updates_and_images(cave_name="B10", year="2014", images=[], description="""- Dug in 2014
  - Contains a narrow tube suitable for digging
  - More digging required
[HM3, page 61, page 104]""")
create_updates_and_images(cave_name="B10", year="2023", images=["B10-2023-1.jpeg", "B10-2023-2.jpeg"], description="""- Relogged in 2023
  - Didn't go in, but looks interesting from the outside""")

create_updates_and_images(cave_name="B12", year="2022", images=["B12-2022.jpeg"], description="""- Spotted in 2022 from red paint marking
  - No known historical information
  - Red paint can just be seen in bottom of image
  - Appears to be a 10m drop blocked by boulders
  - Digging attempted without tools but little progress made
  - Requires long chisel for entrance widening and oversuit to protect (squeeze not passed due to this)
  - Entrance may need traverse line for protection through squeeze
  - Definitely worth returning with tools""")

create_updates_and_images(cave_name="B20", year="2022", images=["B20-2022.jpeg"], description="""- Re-logged in 2022
  - Good digging potential but hasn't been investigated to the end
  - If it connected with M2, it would give extra height to the system""")

create_updates_and_images(cave_name="B91", year="2023", images=["B91-2023-2.jpeg", "B91-2023-1.jpeg"], description="""- Relogged in 2023
  - Small hole 2m deep looks diggable, can’t see if completely dead. No obvious draft
  - Should rename this as B91 means nothing""")

create_updates_and_images(cave_name="B93", year="2023", images=["B93-2023.jpeg"], description="""- Relogged in 2023
  - Should be renamed as B93 doesn't mean anything
  - Small hole going sideways 2m, no obvious draft but easy digging""")

create_updates_and_images(cave_name="B94", year="2024", images=[], description="""No known information, near JJ2. Location 46°15'16"N 13°45'30"E. In archive "2019_Migoveccavelocations" GPX file.""")
create_updates_and_images(cave_name="B95", year="2024", images=[], description="""No known information, near bone cave. Location 46°15'15"N 13°45'33"E. In archive "2019_Migoveccavelocations" GPX file.""")
create_updates_and_images(cave_name="B96", year="2024", images=[], description="""No known information, near JB and JJ2, a bit down the cliff face. Location 46°15'16"N 13°45'28"E. In the archive "2019_Migoveccavelocations" GPX file.""")
create_updates_and_images(cave_name="B97", year="2024", images=[], description="""No known information. Might be the same as "JB" as it's logged very close. Just south of JJ2. Location 46°15'15"N 13°45'29"E. In archive "2019_Migoveccavelocations" GPX file.""")
create_updates_and_images(cave_name="B98", year="2024", images=[], description="""No known information. About 60m due west from B93. Location 46°15'13"N 13°45'31"E. In archive "2019_Migoveccavelocations" GPX file.""")
create_updates_and_images(cave_name="B99", year="2024", images=[], description="""No known information. About 60m WSW from B98, down the cliff face. Location 46°15'12"N 13°45'28"E. In archive "2019_Migoveccavelocations" GPX file.""")
create_updates_and_images(cave_name="B100", year="2024", images=[], description="""No known information. Was in one of the yellow club GPSs, but not in the archive "2019_Migoveccavelocations" file. Located in the cliff face NW of V10. Added to archive file as of 25/06/2024. Location 46.25597 N, 13.75801 E.""")

create_updates_and_images(cave_name="Bone Cave", year="2022", images=["Bone-2022.jpeg"], description="""- Mentioned in passing in HM1, page 215, but no information provided other than it being near the V10 valley
- 2022
  - Note that Bone on the old GPS coordinates seems to correspond to a random shake hole by the tents, likely a different cave since V10 valley infers it being on the cliff
  - No cave there, only a 1 foot deep hole
  - Image is of the shake hole by the tents""")

create_updates_and_images(cave_name="C1", year="2022", images=["C1-2022.jpeg"], description="""- Area C being the area around the container
- Spotted and logged in 2022
  - No previous entrance marked on the GPS in the area
  - Goes around 10m until a small boulder filled chamber""")

create_updates_and_images(cave_name="C2, Bonsai Pot, Teeny Tiny Rhododendron Pot", year="2023", images=["C2-2023-1.jpeg", "C2-2023-2.jpeg"], description="""- (Re)discovered in 2023
  - Named as there are loads of small Rhododendrons around the entrance
  - Large initial pit with various side chambers
  - Low crawl leads to various small chambers which all die, some after squeezes and short down climbs
  - Short climb up leads to a pitch found to have spits
  - This pitch descends quite a way, splitting into three routes all of which are blocked and die""")

create_updates_and_images(cave_name="C3", year="2023", images=["C3-2023-1.jpeg", "C3-2023-2.jpeg"], description="""- Spotted in 2023
  - Not yet descended
  - No signs of spits, but could have been dropped using naturals""")

create_updates_and_images(cave_name="Coincidence Cave", year="2015", images=[], description="""Discovered in 2015 [HM3, page 113]
  - Initially dug but requires more work""")
create_updates_and_images(cave_name="Coincidence Cave", year="2018", images=[], description="""- Revisited in 2018 (not yet written up)
  - Breakthrough made to series of chambers
- Further explored by the Slovenians after 2018
  - All leads seem to have died, from word of mouth""")
create_updates_and_images(cave_name="Coincidence Cave", year="2022", images=[], description="""- Revisited in 2022
  - End of cave terminates in boulder floor at base of pitch, this would be difficult to dig
  - According to Matti, draft disappears into hole above final pitch, may be diggable
  - Perry has started bolt climbing avens from entrance, first one dead""")
create_updates_and_images(cave_name="Coincidence Cave", year="2023", images=[], description="""Revisited, draughting dig in final chamber looks promising, banged by Fratnik Perry and the gang, needs more work but is source of the draught.""")

create_updates_and_images(cave_name="D1, Desperation 1", year="2023", images=["D1-2023-1.jpeg", "D1-2023-2.jpeg"], description="""- Discovered by David Wilson Jnr
  - Named desperation due to convenient use as an emergency toilet
- Revisited in 2023
  - Doesn't seem to go anywhere, not promising""")

create_updates_and_images(cave_name="DW's 1996 Dig", year="2008", images=[], description="""- Nearby to Royston Vasey, mentioned in HM2 draft page 44, 2008
  - Worth going back down again""")
create_updates_and_images(cave_name="DW's 1996 Dig", year="2022", images=[], description="""- Relocation was attempted in 2022
  - No obvious dig could be found, may be the small filled in hole 5m towards Kuk from Royston Vasey
  - Not promising""")

create_updates_and_images(cave_name="E1", year="2007", images=["e1-2007.jpg", "e1-2007(2).jpg"], description="""- Discovered in 2007 [HM2 draft, page 14]
  - A 25m pitch leading to small windows which require widening to pass""")
create_updates_and_images(cave_name="E1", year="2008", images=["e1-2008.jpg"], description="""- Revisited in 2008 [HM2 draft, page 41]
  - "small crawl" ongoing, but abandoned. Still a lead?
  - "deep deep" crawl not dead, dug out to very tight pitch-head. Surface survey, tied centerline to M2 entrance""")
create_updates_and_images(cave_name="E1", year="2022", images=["E1-2022.jpeg"], description="""- Revisited in 2009 [HM2 2019 draft, page 64]
   - cave approx 30m deep
   - dropped second pitch (capping and chiselling) into chamber, dug in floor to bedrock. Quote: "No better nor worst than capped pitch, but not the stunning lead
we were hoping for." Photos in Slov 2009 folder""")

create_updates_and_images(cave_name="E2", year="2022", images=["E2-2022.jpeg"], description="""- Relogged in 2022
- Visited either 2018 or 2019
  - Ben R remembers poking around it with Janet at some point
  - No obvious leads that he can remember
  - Could be revisited""")

create_updates_and_images(cave_name="E6", year="2007", images=["e6-2007.jpg"], description="""- Photo from 2007, information from filename. Red paint marking "E7" noted. No other info
- Unclear if a cave
- could be revisited""")

create_updates_and_images(cave_name="E7", year="2007", images=["e7-2007.jpg"], description="""- Photo from 2007, information from filename. No other info
- Unclear if a cave
- could be revisited""")

create_updates_and_images(cave_name="E8", year="2022", images=["E8-2022.jpeg"], description="""- Not a cave
  - Logged as "obvcave" on historical GPS files
  - Just an alcove, like K1
  - Small free climb through to the roof""")

create_updates_and_images(cave_name="Gelateria, B92", year="2019", images=[], description="""- Logged as B92 on old GPS data (not sure when)
- Uncovered in 2019 due to low snow levels
  - Dug a little but not very promising
  - It is a real cave but get's very tight very quickly and the digging could continue for a long time""")
create_updates_and_images(cave_name="Gelateria, B92", year="2023", images=["Gelateria-2023-1.jpeg", "Gelateria-2023-2.jpeg"], description="""- Poked again in 2023
  - Christened Gelateria since it's next to where we get snow for the bivi
  - Still doesn't look promising, but recovered some digging things left from 2019""")

create_updates_and_images(cave_name="Gondolin", year="2017", images=[], description="""- Discovered in 2017 [HM3, page 232]
  - Pushed for three pitches before the cave dies""")

create_updates_and_images(cave_name="Hare Cave", year="2004", images=[], description="""- Description in HM1, page 183, 2004
  - Was guarded by a large hare in 2003, so left until 2004
  - Draughting strongly from the floor
  - Seems to be a false floor at the top of a rift, could be dug
  - Small phreatic "torpedo" tubes found in upper levels""")
create_updates_and_images(cave_name="Hare Cave", year="2005", images=[], description="""- Revisited in 2005 [HM1, page 191]
  - Torpedo tubes all found to close down, no leads left""")
create_updates_and_images(cave_name="Hare Cave", year="2006", images=[], description="""- Revisited in 2006 [HM1, page 206]
  - Investigated a phreatic tube but it didn't seem to go anywhere
  - A strongly draughting dig in the centre of the chamber but more digging is required to see if this goes or not""")
create_updates_and_images(cave_name="Hare Cave", year="2017", images=[], description="""- Revisited in 2017 [HM3, page 231]
  - As previously reported, the "torpedo tubes" do not go and no more leads were found""")
create_updates_and_images(cave_name="Hare Cave", year="2022", images=["Hare-2022.jpeg"], description="""Relogged and photo taken.""")

create_updates_and_images(cave_name="Jailbreak", year="2013", images=[], description="""- Discovered in 2013 [HM3, page 34]
  - Although it leads to a chamber with many ways on, all of them seem to die
  - Perhaps possible to dig, but no easy places""")
create_updates_and_images(cave_name="Jailbreak", year="2014", images=[], description="""- Revisited and killed in 2014 [HM3, page 58, page 104]
  - Survey and description in HM3 reference
  - Main lead was killed after hauling a boulder out of the way, other lead was also killed""")

create_updates_and_images(cave_name="Janet's Little Cave, MLC, My Little Cave", year="2004", images=[], description="""- Description in HM1, page 184, 2004
  - Small airy passage in the side of a shakehole near the Kuk path
  - Tube from first chamber into second chamber that hasn't been pushed
  - Ongoing lead as of 2004""")
create_updates_and_images(cave_name="Janet's Little Cave, MLC, My Little Cave", year="2009", images=["MLC-2009.jpg", "MLC-2009(2).jpg"], description="""""")
create_updates_and_images(cave_name="Janet's Little Cave, MLC, My Little Cave", year="2019", images=[], description="""- Explored and surveyed in 2019 by Tanguy Racine
  - This was when the Disto data was added to the GitHub repo
  - Presumably found to be dead? No indication of any leads""")
create_updates_and_images(cave_name="Janet's Little Cave, MLC, My Little Cave", year="2022", images=["MLC-2022.jpeg"], description="""- Revisited by DPW and Leo Antwis 2022
  - Dug further but slow progress and not promising
  - Also found cave to be dead, after checking thoroughly for leads (including through tight hole)
  - Only revisited because Ben R forgot he went here in 2019, which is why we need this atlas!!!!""")

create_updates_and_images(cave_name="JCC", year="2008", images=[], description="""- Unsure if this is a cave or not, a GPS waypoint of this name is mentioned in HM2 draft, page 44, 2008
  - East of M19 there is large entrance with a draughting dig that looks promising""")
create_updates_and_images(cave_name="JCC", year="2023", images=["JCC-2023-1.jpeg", "JCC-2023-2.jpeg"], description="""- Relogged in 2023
  - This description does indeed hold true, but dig wasn't investigated""")

create_updates_and_images(cave_name="JJ1", year="2006", images=[], description="""- Discovered in 2006, HM1 page 214
  - Multiple caves in the V8 valley, unclear which is JJ1 from HM1 description alone
  - Seems to be a small chamber, a blowing aven and an unpushed wet crawl""")

create_updates_and_images(cave_name="JJ2, Pella's Feathers", year="2006", images=[], description="""- Discovered in 2006, HM1 page 214
  - Many caves in the V9 valley, but this one is an obvious triangular hole
  - A crawl continues for 10m before splitting
  - The left branch dies, whilst the right branch ends with a rock that requires clearing to progress further""")

create_updates_and_images(cave_name="JJ3, Hawk Cave", year="2006", images=[], description="""- Discovered in 2006, HM1 page 216
  - Initially discovered by Andrej Fratnik and "the other Andrej" back in the 70s
  - Rough survey and description given in HM1 (see above page)
  - Many drafting leads left due to lack of digging equipment and over suits""")
create_updates_and_images(cave_name="JJ3, Hawk Cave", year="2007", images=[], description="""- Revisited in 2007 [HM2 draft, page 15]
  - Left ongoing due to lack of rope and time""")

create_updates_and_images(cave_name="JJ4", year="2006", images=[], description="""- Discovered in 2006, HM1 page 215
  - A winder on the side of Kuk
  - No indication of any leads""")

create_updates_and_images(cave_name="JJ5", year="2006", images=[], description="""- Discovered in 2006, HM1 page 215
  - Crawls and chambers end in a desperate dig beneath hanging death""")

create_updates_and_images(cave_name="JJ6", year="2006", images=[], description="""- Not a cave, but a waypoint set in 2006, HM1 page 215
  - Many caves are visible from this point""")

create_updates_and_images(cave_name="K1, The Goat Shelter", year="2022", images=["K1-2022.jpeg"], description="""- An obvious entrance on the flank of Kuk that only goes to a higher entrance with no leads [HM1, page 195, 2005]""")

create_updates_and_images(cave_name="K2", year="2005", images=[], description="""- Discovered in 2005 [HM1, page 195]
  - Easy digging and massive draught
  - Rock is very loose
  - Two days of digging in 2005""")
create_updates_and_images(cave_name="K2", year="2012", images=[], description="""- Revisited in 2012 [HM2 draft, page 136]
  - Seems to have the most potential of all the area K caves
  - Much digging required before it goes""")
create_updates_and_images(cave_name="K2", year="2015", images=[], description="""- Revisited in 2015 [HM3, page 140]
  - Found to "not go" although given it was an existing dig, this may have been subjective""")
create_updates_and_images(cave_name="K2", year="2022", images=["K2-2022.jpeg"], description="""- Revisited in 2022
  - Goes into small chamber with dig off to the side
  - Possible to dig in crawl sloping up towards the surface
  - No draft felt on a very warm and still day
  - Not very promising""")

create_updates_and_images(cave_name="K3", year="2005", images=[], description="""A large 30m shaft on top of Zeleni Vrh, believed bottomed in the early 90s [HM1, page 195, 2005]""")

create_updates_and_images(cave_name="K4", year="2005", images=[], description="""Entrance on north face of Kuk, found to just be a window with light filtering in from above [HM1, page 195, 2005]""")

create_updates_and_images(cave_name="K5", year="2005", images=[], description="""Entrance on west end of north Kuk face [HM1, page 195, 2005]
  - Abseiled down to scree floor but unclear if cave is completely dead""")

create_updates_and_images(cave_name="K6", year="2005", images=[], description="""- First mentioned 2005
  - Sharp crawl through to mud floor opening out with small draught, could be pushed further with elbow pads [HM1, page 195, 2005]""")
create_updates_and_images(cave_name="K6", year="2012", images=[], description="""- Revisited in 2012 [HM2 draft, page 136]
  - Good progress was made with digging""")
create_updates_and_images(cave_name="K6", year="2022", images=["K6-2022.jpeg"], description="""- Revisited in 2022
  - Closes down into a dig which gets narrower not bigger
  - No obvious draft on a very warm and still day
  - Could be dug further but not very promising""")

create_updates_and_images(cave_name="K7", year="2005", images=["K7-2005-1.jpeg", "K7-2005-2.jpeg"], description="""- A dodgy chamber reached by a 5m clamber over boulders [HM1, page 195, 2005]
  - Small draught, may be possible to dig""")

create_updates_and_images(cave_name="K8", year="2005", images=[], description="""- Overhang into a small choked chamber [HM1, page 195, 2005]""")

create_updates_and_images(cave_name="K9", year="2005", images=[], description="""- Quickly opens out into a 15m pitch [HM1, page 196, 2005]
  - A twin pitch is reachable from a large window
  - Dig potential in a draughting collection of rubble""")
create_updates_and_images(cave_name="K9", year="2022", images=["K9-2022.jpeg"], description="""- Revisited in 2022
  - Located the entrance to photograph and log the GPS location, but pitch not descended""")

create_updates_and_images(cave_name="K10, Torn Testicle", year="2005", images=[], description="""- A very small hole currently blocked but with obvious continuation [HM1, page 196, 2005]
  - Currently only descends 3m
  - stoned rattle when pushed to the north
  - Would need to remove sharp rock blades and lift out boulders in order to progress any further""")
create_updates_and_images(cave_name="K10, Torn Testicle", year="2022", images=["K10-2022.jpeg"], description="""- Revisited in 2022
  - Human sized hole in pavement that's very grabby and unpleasant
  - Person sized alcove at the bottom, with another person sized alcove off to the side at floor level
  - No indication of any continuation or possible digs, not promising""")

create_updates_and_images(cave_name="K11", year="2005", images=[], description="""- Scree climb into a chamber at the end of a shakehole with a cold draught and good digging possibilities [HM1, page 196, 2005]""")
create_updates_and_images(cave_name="K11", year="2012", images=[], description="""- Revisited in 2012, presumed to be reasonably promising but not incredible [HM2 draft, page 136]""")
create_updates_and_images(cave_name="K11", year="2022", images=["K11-2022.jpeg"], description="""- Revisited in 2022
  - Obvious entrance in large shake hole which leads to moderately sized chamber
  - Small descending hole on right side of entrance which may be possible to dig""")

create_updates_and_images(cave_name="K12, Victoria Coach Station", year="2005", images=[], description="""A very large entrance leading to a large snow slope [HM1, page 196, 2005]
  - Enormous draught in places, but many crawls blocked with ice and rubble""")
create_updates_and_images(cave_name="K12, Victoria Coach Station", year="2012", images=[], description="""Revisited in 2012, presumed to be reasonably promising but not incredible [HM2 draft, page 136]""")
create_updates_and_images(cave_name="K12, Victoria Coach Station", year="2015", images=[], description="""- Revisited in 2015 [HM3, page 140]
  - Two days of digging resulted in slow progress
  - Crawl get's narrower
  - Still a "promising" lead""")
create_updates_and_images(cave_name="K12, Victoria Coach Station", year="2022", images=["K12-2022.jpg"], description="""- Revisited in 2022
  - Lead to back of main chamber is a dig continuing in a few meters but with a very loose roof which rocks fell from when crawled through
  - Dig is ongoing but sketchy given looseness of surrounding rock
  - Small alcove found between boulders and snow plug by entrance, no continuation but with more melting this may reveal further alcoves
  - No other leads / crawls could be found, leaving only the scary dig""")

create_updates_and_images(cave_name="K13, Pedestrian Subway / Arrivals Hall", year="2005", images=[], description="""- Rift directly above K12 with possible dig to the South East and rift in floor connecting with the K12 main chamber [HM1, page 197, 2005]""")
create_updates_and_images(cave_name="K13, Pedestrian Subway / Arrivals Hall", year="2022", images=["K13-2022.jpeg"], description="""- Revisited in 2022
  - An infuriatingly good entrance that is completely free of rock and gravel, but unfortunately goes straight back into the roof of K12
  - No indication of any other leads, not promising""")

create_updates_and_images(cave_name="K14", year="2005", images=[], description="""- A shallow rift leading to a small chamber, likely to break onto the surface but not pushed fully [HM1, page 197, 2005]""")
create_updates_and_images(cave_name="K14", year="2022", images=["K14-2022.jpeg"], description="""- Revisited in 2022
  - Rift drops down into boulder filled chamber
  - Route going towards surface
  - Another possible dig underneath the entrance but not easy
  - Overall not promising""")

create_updates_and_images(cave_name="K15, K15i (The Rik-Venn Imaginary Series)", year="2005", images=["K15-2022.jpeg"], description="""- A big blowing and cold cave ending in a dig [HM1, page 197, 2005]
  - Goes in ten meters south and down before reaching a choke underneath a suspended boulder
  - Choke has been dug for one hour but lacked crowbar""")
create_updates_and_images(cave_name="K15, K15i (The Rik-Venn Imaginary Series)", year="2022", images=[], description="""- Revisited in 2022
  - Good entrance which goes into medium sized chamber but quickly chokes
  - Many small birds living in the cave at time of visit""")

create_updates_and_images(cave_name="K16, The Escalator", year="2005", images=["K16-2022.jpeg"], description="""- A series of holes by the Skrbina path [HM1, page 197, 2005]
  - Many alcoves, strong draught and ice plug at bottom""")
create_updates_and_images(cave_name="K16, The Escalator", year="2022", images=[], description="""- Revisited in 2022
  - Can be approached from any direction, but be very careful when approaching from above as the entrance is hidden beneath a small overhang making easy to fall into given the steep scree slope
  - A series of large entrances to a single large chamber with steep scree slope
  - At the top of the slope, a small chamber which is not hard to reach was not investigated as we ran out of time
  - At the bottom of the slope there is a small entrance to another chamber, however the entrance is very small and there is a risk of it being sealed by falling scree upon entry
  - Promising, should be revisited""")

create_updates_and_images(cave_name="K17", year="2005", images=[], description="""- Small chamber at top of shakehole on Podriagora (spelling may be incorrect in HM1?) [HM1, page 197, 2005]
  - Easy digging at end but loose and scary""")

create_updates_and_images(cave_name="K18", year="2005", images=[], description="""- Rift near K17 disappears into a 5m pitch to the floor, not bottomed due to lack of rope and hanging death [HM1, page 197, 2005]""")

create_updates_and_images(cave_name="K19, Coach-Crash Dig", year="2005", images=["K19-2005-1.jpeg", "K19-2005-2.jpeg"], description="""- Entrance rift in a pile of boulders drops down a few meters the runs down into a gravel pile and opens out [HM1, page 198, 2005]
  - Infamous near death experience for Dave Wilson
  - Still possible to access the cave underneath the fallen boulder
  - Boulder now wedged in rift requires splitting, preferably with caps""")
create_updates_and_images(cave_name="K19, Coach-Crash Dig", year="2012", images=[], description="""- Revisited in 2012 [HM2 draft, page 136]
  - Digging continued but no further information""")
create_updates_and_images(cave_name="K19, Coach-Crash Dig", year="2022", images=["K19-2022.jpeg"], description="""- Revisited in 2022
  - Dig looks to have been filled in by falling rock and scree given it's exposed location at the bottom of a large scree slope
  - Gravel chamber is still visible 2 m down, but any digging would result in the surrounding rock collapsing inwards
  - Any dig likely to be filled in within a year or two or scree and rock fall
  - Not promising""")

create_updates_and_images(cave_name="K20, K-Moss", year="2005", images=[], description="""- Near K15, a small mossy hole with a slight draught [HM1, page 198, 2005]
  - Some digging carried out, but would be easy to continue further""")
create_updates_and_images(cave_name="K20, K-Moss", year="2022", images=["K20-2022.jpeg"], description="""- Revisited in 2022
  - Hardly an entrance, just a small mossy depression in the ground with no draught or cold air, on a very warm and still day""")

create_updates_and_images(cave_name="K21", year="2012", images=[], description="""- First mentioned as being revisited in 2012 [HM2 draft, page 136]
  - No further information, but not included in list of "not promising" entrances, so presumed to be reasonably promising""")

create_updates_and_images(cave_name="K22", year="2012", images=[], description="""- First mentioned as being revisited in 2012 [HM2 draft, page 136]
  - 'Not at all promising'""")

create_updates_and_images(cave_name="K23", year="2012", images=[], description="""- First mentioned as being revisited in 2012 [HM2 draft, page 136]
  - No further information, but not included in list of "not promising" entrances, so presumed to be reasonably promising""")

create_updates_and_images(cave_name="K24", year="2012", images=[], description="""- First mentioned as being revisited in 2012 [HM2 draft, page 136]
  - 'Not at all promising'""")

create_updates_and_images(cave_name="K25", year="2012", images=[], description="""- First mentioned as being revisited in 2012 [HM2 draft, page 136]
  - 'Not at all promising'""")

create_updates_and_images(cave_name="K26", year="2012", images=[], description="""- First mentioned as being revisited in 2012 [HM2 draft, page 136]
  - No further information, but not included in list of "not promising" entrances, so presumed to be reasonably promising""")

create_updates_and_images(cave_name="K27", year="2022", images=["K27-2022.jpeg"], description="""- First logged in 2022
  - Logged after thinking it was K14
  - Short drop into gravel filled chamber
  - No continuation or possible digs, not promising""")

create_updates_and_images(cave_name="K28", year="2022", images=["K28-2022.jpeg"], description="""- First logged in 2022
  - A fairly small, draughting, rubble-filled hole about a meter cubed
  - Likely to have been snow covered previously given how sheltered and cold it is
  - Considering draught was felt on a very warm still day, this is a promising indication of nearby cave passage
  - No continuation or digging potential, not promising as a lead in itself""")

create_updates_and_images(cave_name="K29", year="2022", images=["K29-2022.jpeg"], description="""- Previously logged as K99 on the old GPS files, but no mention of it in any hollow mountain editions so unclear why it broke with the area K naming convention
- Logged as K29 in 2022
  - A small chamber 5m underneath large boulders
  - Leads to a second small alcove
  - Not promising as a lead in itself, but cold
  - No obvious draught on a very warm and still day""")

create_updates_and_images(cave_name="Keyhole Cave", year="2023", images=[], description="""- Very close to Postcard Cave, see Postcard cave entry for description of location""")

create_updates_and_images(cave_name="LP1", year="2022", images=["LP1-2022.jpeg"], description="""- Discovered in 2022
  - LP = Limestone Pavement, as previously caves here were named TR1,2,3 etc. but this system breaks down when anyone other than Tanguy finds a cave
  - Small chamber filled with rocks which can be reached from multiple entrances
  - Additional entrance has obvious cave below, likely providing a better entrance but blocked by large rocks
  - Initial hauling and digging of this better entrance
  - Inside loose slope leads to snow plug with crawl going around corner beyond, not pushed as no oversuit available
  - No breakthrough made - good potential, worth revisiting""")

create_updates_and_images(cave_name="LP2", year="2022", images=["LP2-2022.jpeg"], description="""- Discovered in 2022
  - 2m descent to snow plug
  - Draught disappears in front and to the left in a small rift not possible to dig
  - Snow tasted of high quality, according to Leo""")
create_updates_and_images(cave_name="LP2", year="2023", images=[], description="""- Revisited in 2023 by Julien
  - Small tunnel to left which rocks bounce down for a while
  - Potentially leads to pitch""")

create_updates_and_images(cave_name="LP3", year="2022", images=["LP3-2022.jpg"], description="""- Discovered in 2022
  - Not descended, no notes taken at the time
  - May be worth revisiting""")
create_updates_and_images(cave_name="LP3", year="2023", images=[], description="""- Killed in 2023
  - Bolted and descended about 10m down on single bolt backed up with natural
  - Snow plug about 5m down with pitch continuing to the side of it
  - Passage becomes choked and grabby
  - Slight draft but small and too difficult to dig, effectively dead""")

create_updates_and_images(cave_name="LP4", year="2022", images=["LP4-2022.jpeg"], description="""- Discovered 2022
  - May have been descended before
  - Likely plugged by snow
  - Unclear from notes whether this is worth returning to""")
create_updates_and_images(cave_name="LP4", year="2023", images=[], description="""- Visited 2023
  - Didn't go down but stone rattles down a long way
  - Quite promising""")

create_updates_and_images(cave_name="LP5", year="2022", images=["LP5-2022.jpeg"], description="""- Discovered 2022
  - Big hole with gravel and snow at the bottom
  - Likely connects with LP4
  - Unclear from notes whether this is worth returning to""")

create_updates_and_images(cave_name="LP6", year="2022", images=["LP6-2022.jpeg"], description="""- Discovered in 2022
  - Hole in side of shake hole blocked by boulder with visible slope underneath
  - Looks possible to widen the entrance
  - Worth revisiting
  - Also logged as TJ1""")

create_updates_and_images(cave_name="LP7, Gardeners of the Galaxy, The Jaws of Death", year="2022", images=["LP7-2022.jpeg"], description="""- Discovered in 2022
  - A new hole had opened up in a shake hole since the previous expedition before covid
  - Far too loose to descend this year, so instead we chucked a load of rocks down it (many good videos)
  - These are the jaws of death that nearly ate Rory Rose""")
create_updates_and_images(cave_name="LP7, Gardeners of the Galaxy, The Jaws of Death", year="2023", images=[], description="""Full with snowplug, some people walked on the snow but couldn't get down any further, and walls still looked terrifying.""")

create_updates_and_images(cave_name="LP8", year="2022", images=["LP8-2022.jpeg"], description="""- Discovered in 2022
  - Not descended but look interesting given entrance is recently exposed above snow plug
  - Worth revisiting, since new entrances may have been revealed""")
create_updates_and_images(cave_name="LP8", year="2023", images=[], description="""- Revisited in 2023
  - A snow crawl was found but it didn't go very far
  - Rest of the cave seems dead, but high snow levels""")

create_updates_and_images(cave_name="LP9", year="2022", images=["LP9-2022.jpeg"], description="""- Discovered in 2022
  - Only discovered this cave after leaving the mountain
  - Had initially thought this was PF10 but apparently not, since actual entrance was instead logged as M24, and this entrance as PF10
  - The "M24" entrance, which is larger and much further down the pavement has been confirmed as PF10
  - Rocks fall a fair way, definitely worth a look
  - Next to and just above Alex Seaton's old dig into the side of a cliff""")
create_updates_and_images(cave_name="LP9", year="2023", images=[], description="""- Revisited in 2023
  - Entrance is full of snow so can't hear rocks fall down this year
  - Neither Arun or Astrid could fit into the cave, as entrance is much smaller than it looks
  - Would need some widening for anyone to investigate further""")

create_updates_and_images(cave_name="LP10", year="2023", images=["LP10-2023-1.jpeg", "LP10-2023-2.jpeg"], description="""- Discovered 2023
  - Draughting slightly outwards
  - Could be dug but would require more effort than it's worth
  - Very cold""")

create_updates_and_images(cave_name="LP11", year="2023", images=["LP11-2023-1.jpeg", "LP11-2023-2.jpeg"], description="""- Discovered in 2023
  - Draughting out slightly
  - Tight rift continues down for about 4m but not investigated due to lack of over suits""")

create_updates_and_images(cave_name="LP12", year="2023", images=["LP12-2023-1.jpeg", "LP12-2023-2.jpeg"], description="""- Discovered 2023
  - Opens out to a 3m x 1m chamber at the bottom
  - Possibly possible to dig, but very choked""")
create_updates_and_images(cave_name="LP13", year="2023", images=["LP13-2023-1.jpeg", "LP13-2023-2.jpeg"], description="""- Logged in 2023
  - Can see a continuation past the snow at the bottom of a 5m shake hole
  - Not descended due to lack of gear""")
create_updates_and_images(cave_name="LP14", year="2023", images=["LP14-2023-1.jpeg", "LP14-2023-2.jpeg"], description="""- Discovered and killed in 2023
  - Reasonably sized shake hole but dies quickly
  - Free climbed down
  - Dead, completely choked with gravel""")
create_updates_and_images(cave_name="LP15", year="2023", images=["LP15-2023-1.jpeg", "LP15-2023-2.jpeg", "LP15-2023-3.jpeg"], description="""- Discovered and killed in 2023
  - Right next to the jaws of death
  - Initially very promising but found to link to next shake hole after 10m
  - Extensive digging allows for a cracking through trip
  - No obvious continuation, killed""")
create_updates_and_images(cave_name="LP16, Icehorn", year="2023", images=["LP16-2023.jpeg"], description="""- Discovered in 2023
  - Squeezing past the snow plug leads to a large chamber 5m
  - Many small holes leading off of the main shake hole which all seem to drop into the main chamber when throwing rocks down
  - Dig at bottom of large chamber leads to an ongoing crawl but dies quickly in all directions after being dug
  - Another dig up in the left wall of the chamber dies quickly
  - All other leads seem to die""")
create_updates_and_images(cave_name="LP17", year="2023", images=["LP17-2023-1.jpeg", "LP17-2023-2.jpeg"], description="""- Discovered and killed in 2023
  - Twin holes, climb down one and then stoop into the other
  - Looked promising and descended down for about 5m but dies
  - No obvious digs or drafts""")
create_updates_and_images(cave_name="LP18", year="2023", images=["LP18-2023-1.jpeg", "LP18-2023-2.jpeg"], description="""- Discovered in 2023
  - Not descended but possible continuation out of sight
  - Not that promising but worth a look""")
create_updates_and_images(cave_name="LP19", year="2023", images=["LP19-2023-1.jpeg", "LP19-2023-2.jpeg"], description="""- Discovered in 2023
  - 7m hole with possible continuation out of sight
  - Not descended as looked at other holes instead""")
create_updates_and_images(cave_name="LP20", year="2023", images=["LP20-2023-1.jpeg", "LP20-2023-2.jpeg"], description="""- Discovered in 2023
  - Previously logged as TJRIFT
  - Large 10m deep rift definitely worth a look
  - Two entrances and another two holes just down the valley which may also be connected""")
create_updates_and_images(cave_name="LP21, Tin Can Hole", year="2023", images=["LP21-2023.jpeg"], description="""- Discovered and killed in 2023
  - Dropped previously since we found a fish tin at the bottom
  - No obvious digs all completely full of rubble 
  - Dropped on a rope but may have been descended by crazy free climb before, no spits seen""")
create_updates_and_images(cave_name="LP22", year="2023", images=["LP22-2023.jpeg"], description="""- Discovered in 2023
  - Small dark not drafting hole
  - Not investigated since no over suit but worth a look""")
create_updates_and_images(cave_name="LP23", year="2023", images=["LP23-2023.jpeg"], description="""- Discovered and killed in 2023
  - Can walk in
  - Dead, completely choked with gravel""")
create_updates_and_images(cave_name="LP24, (Arun's) Secret Hole, How much is that Chamois in the window", year="2023", images=["LP24-2023-1.jpeg", "LP24-2023-2.jpeg", "LP24-2023-3.jpeg"], description="""- Discovered and killed in 2023
  - Dropped by Astrid and Arun
  - Incredibly tight and sharp, required much bashing and multiple bolts
  - Dead chamois found on one ledge halfway down
  - Cave chokes up with gravel, no obvious digs or drafts
  - An impressive ice stal at the bottom""")
create_updates_and_images(cave_name="LP25", year="2023", images=["LP25-2023.jpeg"], description="""- Discovered in 2023
  - Not investigated since focussing on other caves
  - Very close to JOD
  - Worth a free climb down, quick to check""")
create_updates_and_images(cave_name="LP26", year="2023", images=["LP26-2023.jpeg"], description="""- Discovered in 2023
  - Not investigated since focussing on other caves
  - Very close to JOD and next to LP25
  - Worth a free climb down, quick to check""")
create_updates_and_images(cave_name="LP27", year="2023", images=["LP27-2023-1.jpeg", "LP27-2023-2.jpeg"], description="""- Discovered in 2023
  - Snow plugged rift not descended
  - Doesn't look that promising""")
create_updates_and_images(cave_name="LP28, Christmas Pot", year="2023", images=["LP28-2023-1.jpeg", "LP28-2023-2.jpeg"], description="""- Discovered in 2023
  - Right next to the LP Christmas tree
  - Dies in boulders at the bottom
  - Possible dig but large overlying boulders mean probably not worthwhile""")
create_updates_and_images(cave_name="LP29", year="2023", images=["LP29-2023.jpeg"], description="""- Discovered in 2023
  - Not descended but looks like a possible dig at the bottom
  - Likely free climbable""")
create_updates_and_images(cave_name="LP30", year="2023", images=["LP30-2023.jpeg"], description="""- Discovered in 2023
  - Not descended but looks like a possible dig at the bottom
  - Likely free climbable""")
create_updates_and_images(cave_name="LP31", year="2023", images=["LP31-2023-1.jpeg", "LP31-2023-2.jpeg"], description="""- Discovered in 2023
  - Not descended but small chamber at the bottom with snow
  - Needs bolting""")
create_updates_and_images(cave_name="LP32", year="2023", images=["LP32-2023-1.jpeg", "LP32-2023-2.jpeg"], description="""- Discovered in 2023
  - Not descended but small chamber beneath a boulder""")
create_updates_and_images(cave_name="LP33", year="2023", images=["LP33-2023.jpeg"], description="""- Discovered in 2023
  - Not descended possible dig at the bottom
  - Needs bolting""")
create_updates_and_images(cave_name="LP34", year="2023", images=["LP34-2023.jpeg"], description="""- Discovered in 2023
  - Not descended since needs bolting
  - Large hole with ice at the bottom""")
create_updates_and_images(cave_name="LP35", year="2023", images=["LP35-2023.jpeg"], description="""- Discovered in 2023
  - Screen covered floor 4m down
  - Not descended""")
create_updates_and_images(cave_name="LP36", year="2023", images=["LP36-2023.jpeg"], description="""- Discovered in 2023
  - Pot with stream-way like walls
  - Bottom is filled with scree, 3-4m deep
  - Undescended""")
create_updates_and_images(cave_name="LP37, Stalagm-ice", year="2023", images=["LP37-2023-1.jpeg", "LP37-2023-2.jpeg"], description="""- Discovered and killed in 2023
  - Obvious opening drops down to choked shaft
  - Small but impressive ice formations at back of chamber
  - No obvious digs or ways on""")

create_updates_and_images(cave_name="M1", year="1983", images=[], description="""- Depth in 1983 recorded as -61m [HM1, page 13]""")
create_updates_and_images(cave_name="M1", year="2007", images=[], description="""- Revisited in 2007 [HM2 draft, page 13]
  - Repushed and resurveyed
  - One ongoing lead left in a window, which may require some chemical persuasion""")
create_updates_and_images(cave_name="M1", year="2022", images=["M1-2022.jpeg"], description="""""")

create_updates_and_images(cave_name="M2", year="2022", images=["M2-2022.jpeg"], description="""- An entrance to the system. Discovered by the Slovenes back in the 70s [HM1, page 6]""")

create_updates_and_images(cave_name="M3", year="1993", images=[], description="""- No known coordinates, altitude or depth as of 1993 [HM1, page 13]""")
create_updates_and_images(cave_name="M3", year="2023", images=[], description="""- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)""")

create_updates_and_images(cave_name="M4", year="1993", images=[], description="""- No known depth as of 1993 [HM1, page 13]""")
create_updates_and_images(cave_name="M4", year="2022", images=["M4-2022.jpeg"], description="""- Relogged in 2022
  - Not descended
  - M4 painted in blue, rather than red paint for some reason
  - May be worth revisiting, since close to the bivi""")

create_updates_and_images(cave_name="M5", year="1993", images=[], description="""- No known coordinates, altitude or depth as of 1993 [HM1, page 13]""")
create_updates_and_images(cave_name="M5", year="2023", images=[], description="""- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)""")

create_updates_and_images(cave_name="M6", year="1993", images=[], description="""- Depth indicated as -67 m as of 1993 [HM1, page 13]""")
create_updates_and_images(cave_name="M6", year="2007", images=["M6-2007.jpg"], description="""- Revisited 2007 [HM2 draft, page 13]
  - small extensions - "pretty little bit of stream formed new cave, ended in draughting bedding plane dig."
  - images of surface shaft and Tetley putting in a rebelay
  - resurveyed""")
create_updates_and_images(cave_name="M6", year="2023", images=[], description="""- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)""")

create_updates_and_images(cave_name="M7", year="1993", images=[], description="""- Depth indicated as -20m as of 1993 [HM1, page 13]""")
create_updates_and_images(cave_name="M7", year="2022", images=['M7-2022.jpeg'], description="""- Relogged in 2022""")

create_updates_and_images(cave_name="M8", year="1993", images=[], description="""- Depth indicated as -13m as of 1993 [HM1, page 13]""")
create_updates_and_images(cave_name="M8", year="2022", images=["M8-2022.jpeg"], description="""- Relogged in 2022""")

create_updates_and_images(cave_name="M9", year="1993", images=[], description="""- Depth indicated as -19m as of 1993 [HM1, page 13]""")
create_updates_and_images(cave_name="M9", year="2022", images=["M9-2022.jpeg"], description="""- Relogged in 2022""")

create_updates_and_images(cave_name="M10", year="1993", images=[], description="""- Depth indicated as -33m as of 1993 [HM1, page 13]
- Visited frequently to collect snow for water""")
create_updates_and_images(cave_name="M10", year="2015", images=[], description="""- Explored more thoroughly in 2015 [HM3, page 111]
  - One way on dies in a too-tight rift
  - Another route dies in an icy section""")
create_updates_and_images(cave_name="M10", year="2022", images=["M10-2022.jpeg"], description="""""")

create_updates_and_images(cave_name="M11", year="1993", images=[], description="""- Depth indicated as -6m as of 1993.
  - "It is difficult to understand why holes such as M11 (6m deep!) were noted, as the plateau has many similar or deeper surface holes which were not then formally recognised." [HM1, page 13]""")
create_updates_and_images(cave_name="M11", year="2023", images=[], description="""- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)""")

create_updates_and_images(cave_name="M12 maybe?", year="1993", images=[], description="""- Depth indicated as -15m as of 1993 [HM1, page 13]""")
create_updates_and_images(cave_name="M12 maybe?", year="2022", images=["M12 maybe-2022.jpeg"], description="""- Possibly relocated 2022
  - Not descended but found a cave which looks dug out, and is in the vicinity of M12 on the GPS
  - At the exact location of M12 on the GPS, there are no obvious entrances
  - M12 could be off the cliff, since some old coordinates are dozens of meters away from their actual location
  - Should definitely be investigated, can hear a small chamber when rocks thrown""")

create_updates_and_images(cave_name="M13", year="1993", images=[], description="""- Depth indicated as -15m as of 1993 [HM1, page 13]""")
create_updates_and_images(cave_name="M13", year="2023", images=[], description="""- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)""")

create_updates_and_images(cave_name="M14", year="1993", images=[], description="""- Depth indicated as -22m as of 1993 [HM1, page 13]""")
create_updates_and_images(cave_name="M14", year="2023", images=[], description="""- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)""")

create_updates_and_images(cave_name="M15", year="1993", images=[], description="""- Depth indicated as -58m as of 1993 [HM1, page 13]""")
create_updates_and_images(cave_name="M15", year="2022", images=["M15-2022.jpeg"], description="""- Relogged 2022""")

create_updates_and_images(cave_name="M16", year="1982", images=[], description="""- An entrance to the main system discovered by the Slovenes in 1982. [HM1, page 8]""")
create_updates_and_images(cave_name="M16", year="2022", images=["M16-2022.jpeg"], description="""""")

create_updates_and_images(cave_name="M17", year="1993", images=[], description="""- Depth indicated as -58m as of 1993 [HM1, page 13]""")
create_updates_and_images(cave_name="M17", year="1994", images=[], description="""- Further explored in 1994, HM1 page 25 with survey
  - Pushed until it got too cold, going lead at bottom of cave""")
create_updates_and_images(cave_name="M17", year="2018", images=["M17-2018.jpg"], description="""- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)""")

create_updates_and_images(cave_name="M18, Torn T-Shirt Pot, Jama Strgane Srajce", year="1994", images=[], description="""- Large entry about discovery in HM1 page 19 from 1994
- An entrance into the main system""")
create_updates_and_images(cave_name="M18, Torn T-Shirt Pot, Jama Strgane Srajce", year="2022", images=["M18-2022.jpeg"], description="""""")

create_updates_and_images(cave_name="M19, BS Pot", year="1994", images=[], description="""- Entry about discovery in HM1 page 25 from 1994
  - Extremely tight and loose
  - Next to M17
  - Incredibly strong draft
  - Pushing stopped due to incredibly loose walls requiring shoring""")
create_updates_and_images(cave_name="M19, BS Pot", year="2023", images=[], description="""- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)""")

create_updates_and_images(cave_name="M20, White Shiver Pot", year="1994", images=[], description="""- Large entry about discovery and survey in HM1, page 17, 1994
  - Large 15m pothole with 20m drop to a snow plug
  - Squeezing past the snow plug led to a large chamber
  - No indication of continuation of digging potential""")
create_updates_and_images(cave_name="M20, White Shiver Pot", year="2023", images=[], description="""- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)""")

create_updates_and_images(cave_name="M21, B9, Jackie's Blower", year="1995", images=[], description="""- Entry about discovery and survey in HM1, page 33, 1995
  - Two entrances near the top of the cliff
  - Very loose""")
create_updates_and_images(cave_name="M21, B9, Jackie's Blower", year="2016", images=[], description="""- Revisited in 2016 [HM3, page 219]
  - Has apparently been visited a number of times since initial discovery
  - The bottom pitch was pushed but no mention of whether this lead is ongoing or not""")

create_updates_and_images(cave_name="M22, Venus Cave", year="1995", images=[], description="""- Entrance about discovery and survey in HM1, page 34, 1995
  - Named as it sucked flies in like a venus fly trap, strong draft!""")
create_updates_and_images(cave_name="M22, Venus Cave", year="2018", images=[], description="""- Explored in 2018 as seen in 2018 documentary video, no encouraging ways on after digging""")
create_updates_and_images(cave_name="M22, Venus Cave", year="2022", images=["M22-2022.jpeg"], description="""""")

create_updates_and_images(cave_name="M23, Julie's Pantry Cave", year="1995", images=[], description="""- Entry on discovery and survey in HM1, page 32 from 1995
  - Very strong draft
  - Terminates at a small passage""")
create_updates_and_images(cave_name="M23, Julie's Pantry Cave", year="2023", images=[], description="""- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)""")

create_updates_and_images(cave_name="M24, PF10", year="1995", images=["PF10Survey.jpeg"], description="""- Entry about discovery and survey in HM1, page 36, 1995
  - Crawl at bottom deemed too unstable to continue exploration
  - Abandoned for other leads in 1995
  - By the looks of EE, still a window on P4 and a bolt climb at bottom of P4 left unexplored. Worth a look. Could break into southern extensions.""")
create_updates_and_images(cave_name="M24, PF10", year="2022", images=["M24-2022.jpeg"], description="""- Relocated in 2022
  - Previously M24 and PF10 were logged as being over 100m apart
  - Upon throwing rocks down, sounds of ice smashing could be heard""")
create_updates_and_images(cave_name="M24, PF10", year="2023", images=[], description="""Began rebolting but the storms prevented us reaching th bottom of the first pitch. Lots of scary ice.""")

create_updates_and_images(cave_name="M25, Gulliver's Kipper Cave", year="1995", images=[], description="""- Entry about discovery in HM1, page 33, 1995
  - On the face of Mig, visible from the shepherd's huts
  - Initially opens up but all proceeding chambers closed down
  - Little effort put into the cave, no survey taken. Could be worth a revisit for the katasterjam.""")
create_updates_and_images(cave_name="M25, Gulliver's Kipper Cave", year="2019", images=[], description="""- Rediscovered by Ben R and Ben H in 2019. It is located up the coincidence valley and is a very big entrance with a P10 to a pile of wood. Was bolted in 1996 and is more or less dead (see HM1)
- Coordinates converted from GK5 to Lat/Long: 46.245408, 13.765006
- May have digging potential""")

create_updates_and_images(cave_name="Monatip", year="2007", images=[], description="""An entrance to the main system discovered in 2007 [HM2 draft, page 18]""")

create_updates_and_images(cave_name="Moth Cave", year="2006", images=[], description="""- Discovered in 2006 [HM1, page 207]
  - Located close to Hare cave
  - Left at a pile of draughting boulders which are easy and quick to dig
  - About an hour of digging was carried out in 2006""")
create_updates_and_images(cave_name="Moth Cave", year="2007", images=["Moth Cave-2007.jpg"], description="""- Revisited in 2007 [HM2 draft, page 15, page 25]
  - Large volumes of gravel removed
  - All leads seem to have been killed
  - Only way on is another squeeze which would require rock removal to pass""")
create_updates_and_images(cave_name="Moth Cave", year="2022", images=["Moth-2022.jpeg"], description="""""")

create_updates_and_images(cave_name="Mugo", year="2020", images=["Mugo Upper Entrance-2020.jpg", "Mugo Lower Entrance-2020.jpg"], description="""- Discovered in 2020 [Slovenia subsite, 2020 Summer Action]
  - accessed from the contour path linking Kal to the head of the Tolminka valley
  - found while looking at overhangs and possible cave entrances that had been identified from below before ascending scree slopes below Gondolin
  - short descent unveiled a chamber which continued with a small climb into a dead end
  - two entrances
  - surveyed; final length 38m, depth 7m""")

create_updates_and_images(cave_name="N01", year="2014", images=[], description="""- Not the same as the N1 in area N
- Rediscovered in 2014 [HM3, page 104]
  - Found to be 3m deep and leading into a small chamber but no sign of downward progression""")
create_updates_and_images(cave_name="N01", year="2023", images=["N01-2023-1.jpeg", "N01-2023-2.jpeg", "N01-2023-3.jpeg"], description="""- Relogged in 2023
  - Not dropped, red spray paint and gravel floor visible""")

create_updates_and_images(cave_name="N1", year="2023", images=[], description="""- No known information
- Note that this is different to the N01 found to the south of Kuk""")

create_updates_and_images(cave_name="N2", year="2023", images=[], description="""No known information""")
create_updates_and_images(cave_name="N3", year="2023", images=[], description="""No known information""")
create_updates_and_images(cave_name="N4", year="2023", images=[], description="""No known information""")
create_updates_and_images(cave_name="N5", year="2023", images=[], description="""No known information""")
create_updates_and_images(cave_name="N6", year="2023", images=[], description="""No known information""")
create_updates_and_images(cave_name="N7", year="2023", images=[], description="""No known information""")
create_updates_and_images(cave_name="N8", year="2023", images=[], description="""No known information""")

create_updates_and_images(cave_name="N9, Kuk Pot", year="2012", images=["N9, Kuk Pot-2012.jpg"], description="""- First mentioned in 2012 [HM2 draft, page 136]
  - First two pitches dropped
  - Continuation visible but left due to lack of time""")
create_updates_and_images(cave_name="N9, Kuk Pot", year="2019", images=[], description="""- Returned to and killed in 2019. Writeup on UKCaving [here](https://ukcaving.com/board/index.php?threads/maraton-iccc-jspdt-slovenia-expedition-july-2019.25199/).""")

create_updates_and_images(cave_name="N10", year="2023", images=[], description="""No known information""")

create_updates_and_images(cave_name="N11", year="2023", images=[], description="""- Someone (unknown) recently logged this on one of the club GPS's. Coords are: 46°16'02"N 13°45'38"E
- Worth having a look""")

create_updates_and_images(cave_name="N12", year="2023", images=[], description="""Someone (unknown) recently logged this on one of the club GPS's. Coords are: 46°15'47"N 13°45'33"E""")

create_updates_and_images(cave_name="N13", year="2023", images=[], description="""Also logged as "BOLTCAVE" and "WB1". A vertical shaft with spits. No information on who originally descended this, no mentions in hollow mountains. Definitely worth checking out. Coords are: 46°15'47"N 13°45'37"E""")

create_updates_and_images(cave_name="N14", year="2023", images=[], description="""- Also logged as "WB2". An approximately 5m deep vertical rift with possible cave at the bottom. Remains undescended because of lack of resources. Definitely worth checking out. Coords are: 46°15'44"N 13°45'38"E""")

create_updates_and_images(cave_name="0131", year="2023", images=[], description="""Possibly an accidental GPS marker, so might not be a cave. Between N11 and N12. Coords are: 46°15'54"N 13°45'38"E""")

create_updates_and_images(cave_name="P01 (North of Kuk)", year="2023", images=[], description="""- Included on the area N map, HM3 page 264
- No information regarding this cave was found""")

create_updates_and_images(cave_name="P1", year="2023", images=["P1-2023-1.jpeg", "P1-2023-2.jpeg"], description="""- Discovered in 2023
  - Although very likely to have been discovered earlier, but no records of it
  - Very close to M17, M19, JCC
  - P series for Plateau as all the other caves in this area are either M series, which we don't continue any more, or have random names
  - Short 5m drop to a small chamber, can’t see the bottom with bright sunlight, don’t look easily climbable
  - Not descended due to lack of gear and time""")

create_updates_and_images(cave_name="P2", year="2023", images=["P2-2023.jpeg"], description="""- Discovered in 2023
  - Again, likely looked in before
  - Can walk in but I’d tear my clothes, small chamber can’t see the back. Many fake caves around""")

create_updates_and_images(cave_name="P3", year="2023", images=["P3-2023.jpeg"], description="""- Discovered in 2023
  - Again, likely looked in before
  - A bunch of holes above M17 which you’d need to be clipped in to investigate, but look as though they go back a fair way
  - Potentially one of these is Mouse cave but poorly logged? The GPS for it was a fair way off but didn't correspond to any obvious entrance""")

create_updates_and_images(cave_name="P4", year="2023", images=["P4-2023-1.jpeg", "P4-2023-2.jpeg"], description="""- Discovered in 2023
  - Didn't drop in since no kit
  - Looks like it probably dies but worth a proper look
  - Small chamber may continue round to the left""")

create_updates_and_images(cave_name="P5", year="2023", images=["P5-2023-1.jpeg", "P5-2023-2.jpeg"], description="""- Discovered in 2023
  - Didn't look inside since no kit but so close to cliff abseil that it should be looked at
  - Might be a good dig if nothing else""")

create_updates_and_images(cave_name="Planika", year="2007", images=["Planika-2007(1).jpg"], description="""- Discovered in 2007 [HM2 draft, page 18]
  - Large snow and ice filled cave with holes driven through it by water and air
  - Ends in an un-pushed ice tube after a scary boulder funnel""")
create_updates_and_images(cave_name="Planika", year="2008", images=["Planika-2007(2).jpg"], description="""- Revisited in 2008 [HM2 draft, page 31]
  - No further progress on the route from the previous year made due to high snow levels
  - A new rift (implied name Echo Rift) found from a water drilled hole through the snow plug which was pushed down a pitch
  - Another chamber was found from a window
  - Upper rift 'Diggers of the UG'""")

create_updates_and_images(cave_name="Postcard Cave", year="2004", images=[], description="""- Briefly mentioned in HM1, page 183, 2004
  - In an adjacent shakehole to the east of Hare Cave
  - No further information""")
create_updates_and_images(cave_name="Postcard Cave", year="2007", images=[], description="""- Description from 2007 on club wiki along with keyhole cave:
  - 'Keyhole cave and postcard cave are very near the bivvy. The easiest way to describe them is to start from the edge of the shakehole with hare cave in. Facing towards primadoni direction hare cave is on the left side of the shake hole. Basically you are standing above postcard cave. It is in a smaller shakhole behind you. The entrance is a very low thin bedding plane that immediately breaks into an 'impressive' entrance phreatic tube with a choked draughting rift in the floor. This looks like many other entrances, so the way to confirm is that keyhole cave should also be present in the shakehole. Still facing the same way (i.e. away from the shakehole towards the hare cave shakehole), keyhole cave is on the left side and is almost completely invisible unless you are standing in exactly the right place. Entrance is a lovely phreatic keyhole about 1m high and really very hidden. The cave follows this shape for a couple of metres before ending in a choked small chamber. Not that interesting but a great entrance - would make a superb start to a trip'""")

create_updates_and_images(cave_name="Primadona", year="2000", images=[], description="""- A main entrance to the main system known about for a long time by the Slovenes
- First mentioned in HM1, page 118, 2000""")

create_updates_and_images(cave_name="Pueblo", year="2022", images=["Pueblo-2022.jpeg"], description="""- Not a cave
  - Fratnik's old bivi spot from many decades ago
  - Still filled with metal tent poles
- Relogged in 2022""")

create_updates_and_images(cave_name="Razor Cave", year="2006", images=[], description="""- A cave near the Razor hut, extensively explored by the JSPDT [HM1, page 208, 2006]""")
create_updates_and_images(cave_name="Razor Cave", year="2007", images=[], description="""- Surveyed in 2007 [HM2 draft, page 14]""")

create_updates_and_images(cave_name="Royston Vasey", year="2001", images=[], description="""- Entry about discovery and a survey in HM1, page 147, 2001
  - Very close to the bivi, in an 8m deep shake hole
  - Good digging potential given it's proximity to the bivi""")
create_updates_and_images(cave_name="Royston Vasey", year="2008", images=[], description="""- Rediscovered in 2008 [HM2 draft, page 44]
  - Not pushed any further, only rediscovered""")
create_updates_and_images(cave_name="Royston Vasey", year="2022", images=["Royston_Vasey-2022.jpg"], description="""- Re-bolted and dug in 2022
  - All possible leads investigated
  - Can hear cave continues onto another chamber at bottom of final pitch but not worth blasting since walls close in
  - Conclusively dead this time""")

create_updates_and_images(cave_name="S1, East Pole", year="1998", images=[], description="""- Mentioned in HM1, page 99, 1998
  - Multiple chambers and small pitches, but very unstable
  - Multiple digging trips have since returned to the cave with no major breakthroughs
  - '[a] promising lead'""")
create_updates_and_images(cave_name="S1, East Pole", year="2007", images=["S1-2007.jpg"], description="""- Revisisted in 2007, 2008 and 2009
  - No write ups of the trips; only references to progress from tweet [Votla Gora tweets, 2:32 PM Aug 10th, 2008] given below
  - "S1: way to right hand pitch found by hammering rift above big pitch.Strong draft from tight meander, several hrs hard hammer.4th system?"
  - entrance photos and walk labelled under Eastpole""")
create_updates_and_images(cave_name="S1, East Pole", year="2009", images=["S1-2009.jpg"], description="""""")
create_updates_and_images(cave_name="S1, East Pole", year="2017", images=[], description="""- Revisited in 2017 [HM3, page 271] [2017 logbook]
  - Did not manage to make progress on the upwards dig
  - Marginal progress was made over a tight rift, despite the loss of the lump hammer""")

create_updates_and_images(cave_name="S2", year="2007", images=["S2-2007.jpg"], description="""- Photo of entrance by Jarvist
- No known information""")

create_updates_and_images(cave_name="S3", year="2007", images=["S3-2007.jpg"], description="""- Photo of entrance by Jarvist
- No known information""")

create_updates_and_images(cave_name="S4", year="2023", images=[], description="""- No known information""")

create_updates_and_images(cave_name="S5", year="2007", images=["S5-2007.jpg"], description="""""")
create_updates_and_images(cave_name="S5", year="2013", images=[], description="""- Discovered in 2013 [HM3, page 37]
  - Initially a free-climbable p6, followed by a p20 that requires a rope
  - Appears that this hasn't been descended
  - Confused whether the 2007 image is the same cave as Gergely's description from 2013 HM3""")

create_updates_and_images(cave_name="S6", year="2013", images=[], description="""- Discovered in 2013 [HM3, page 37]
  - Strongly drafting, just as much as S1
  - Requires work but is the most promising lead in the area""")

create_updates_and_images(cave_name="S7", year="2007", images=["S7-2007.jpg", "S7-2007(pitch).jpg"], description="""""")
create_updates_and_images(cave_name="S7", year="2013", images=[], description="""- Discovered in 2013 [HM3, page 37]
  - Cold air drafting from between two boulders
  - Not an actual entrance, but could be dug into one
  - Photo of entrance by Jarvist
  - Confused whether the lablled 2007 images are the same cave as Gergely's description from 2013 HM3 given that the 2007 photos show a pitch. It's not even that clear whether they relate to the same cave as each other, they look a little different""")

create_updates_and_images(cave_name="S8", year="2007", images=["S8-2007.jpg"], description="""- Photo of entrance by Jarvist
- No known information""")

create_updates_and_images(cave_name="S12", year="2022", images=["S12-2022.jpeg"], description="""- Relogged in 2022
  - No known information
  - If this is S12, where are S8-11?""")

create_updates_and_images(cave_name="S-A", year="2022", images=["S-A-2022.jpeg"], description="""- Relogged in 2022
  - No idea what this is, but there's a large shake hole there
  - Likely a nearby cave, but no obvious what the GPS point was referring to
  - Should ask some old(er) lags""")

create_updates_and_images(cave_name="Samo1", year="2022", images=["SAMO1-2022.jpeg"], description="""- Unclear when Samo first logged this
- Revisited in 2022
  - Crawl goes down at least 5m to capped entrance to small chamber
  - Large draft and nearby to new Gardeners of the Galaxy hole, and clearly in the same line of shake holes as it, but cave heads in different direction
  - Not worth revisiting""")

create_updates_and_images(cave_name="Samo2", year="2022", images=["SAMO2-2022.jpeg"], description="""- Unlcear when Samo first logged this
- Revisited in 2022
  - 10m deep shaft, gravel floor with ice plug 2 by 10 m diameter at bottom
  - Big draught and adjacent holes connects back in
  - Cave conclusively killed""")

create_updates_and_images(cave_name="Stag Cave", year="2007", images=[], description="""- Discovered in 2007 [HM2 draft page 15]
  - Within 20m of the tents
  - An initial pitch leads to a dead chamber""")

create_updates_and_images(cave_name="Storm cave", year="2004", images=[], description="""- Description in HM1, page 184, 2004
  - Main way on blocked by snow
  - Other entrances in the area which were blocked by snow""")

create_updates_and_images(cave_name="Sunset Hole", year="2014", images=[], description="""- Discovered in 2014 [HM3, page 104] by Fiona H
  - A small hole, below Sunset Spot, not far off the main path (left when looking uphill) up to the Plateau
  - Short tube sloping downwards into small chamber (room to turn around kneeling), tube continued sloping downwards and got too tight (leg sized)
  - Not a good lead""")

create_updates_and_images(cave_name="T2", year="1998", images=[], description="""- Mentioned in HM1, page 99, 1998
  - A tight strongly drafting rift passage that requires hammering""")

create_updates_and_images(cave_name="The Freezer", year="2001", images=[], description="""- Mentioned in HM1, page 148, 2001
  - Further up the Gardener's World valley
  - One un-pushed lead at the end in a dodgy boulder choke
  - Contained a nice ice stalactite""")
create_updates_and_images(cave_name="The Freezer", year="2023", images=[], description="""- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)""")

create_updates_and_images(cave_name="The Hairdryer", year="1998", images=[], description="""- Mentioned in HM1, page 99, 1998
  - "A slow shuffle into the bedding leads to a small, 8” hole. Beyond this restriction was black space, and the source of the very strong draft"
  - Noted in Jarv's requests for stories in HM2 Doesn't seem to have been investigated further""")

create_updates_and_images(cave_name="TR1", year="2014", images=[], description="""- Discovered in 2014 [HM3, page 62]
  - A 1 x 1.5 m hole plugged with snow
  - Multiple entrances, some of which free climbable and reaches floor of ice and rubble
  - Left undescended""")
create_updates_and_images(cave_name="TR1", year="2016", images=[], description="""- Returned to in 2016 [HM3, page 188]
  - All leads were found to die quickly in rubble and ice""")
create_updates_and_images(cave_name="TR1", year="2022", images=["TR1-2022.jpeg"], description="""""")

create_updates_and_images(cave_name="TR2", year="2014", images=[], description="""- Discovered in 2014 [HM3, page 62]
  - Snow and rubble slope leads down to a low chamber with several blind avens
  - Terminal chamber reached after a short dug out crawl""")
create_updates_and_images(cave_name="TR2", year="2022", images=["TR2-2022.jpeg"], description="""- Revisited in 2022 by David and Rory
  - Near entrance just prior to crawl, on left space noticed under boulder, dug out, leading to chamber
  - Chamber terminates in too tight rift, Aven above chamber leads to undug light from surface
  - Draft disappears underneath climb down into chamber into rock floor
  - Thin person should investigate rift, might be possible to blast""")

create_updates_and_images(cave_name="TR3", year="2014", images=[], description="""- Discovered in 2014 [HM3, page 62]
  - Entrance choke can be down-climbed to a small chamber with continuation requiring rock removal with tape, crowbar and chisel""")
create_updates_and_images(cave_name="TR3", year="2022", images=["TR3-2022.jpeg"], description="""""")

create_updates_and_images(cave_name="TR4", year="2014", images=[], description="""- Discovered in 2014 [HM3, page 63]
  - Obvious dig with upturned boulders near entrance""")
create_updates_and_images(cave_name="TR4", year="2022", images=["TR4-2022.jpeg"], description="""""")

create_updates_and_images(cave_name="TR5", year="2014", images=[], description="""- Discovered in 2014 [HM3, page 63]
  - Small free climb down
  - Draughting with easy digging""")
create_updates_and_images(cave_name="TR5", year="2022", images=["TR5-2022.jpeg"], description="""""")

create_updates_and_images(cave_name="TR6", year="2014", images=[], description="""- Discovered in 2014 [HM3, page 63]
  - Dig in vegetated shakehole
  - Closes down as boulders meet ceiling
  - Harder digging""")
create_updates_and_images(cave_name="TR6", year="2022", images=["TR6-2022.jpeg"], description="""""")

create_updates_and_images(cave_name="TR7", year="2014", images=[], description="""- Discovered in 2014 [HM3, page 63]
  - No obvious digging, both branches quickly become impassable""")
create_updates_and_images(cave_name="TR7", year="2022", images=["TR7-2022.jpeg"], description="""""")

create_updates_and_images(cave_name="TR8", year="2014", images=[], description="""- Discovered in 2014 [HM3, page 63]
  - Ends in rift blocked by ice and cobbles
  - Would need serious digging if ice still present""")
create_updates_and_images(cave_name="TR8", year="2022", images=["TR8-2022.jpeg"], description="""""")
create_updates_and_images(cave_name="TR8", year="2023", images=["TR8-2023-1.jpeg", "TR8-2023-2.jpeg"], description="""- Revisited in 2023
  - New hole likely uncovered from ice plug melt revealed 10m continuation
  - Squeezing through leads to small chamber, second squeeze and then larger 5m wide, 10m long and 5m tall chamber
  - End chamber called Kevin's Wine Cellar, “I’d genuinely put stuff in there, like wine and cheese” -Kevin
  - Drafting quite a lot, and possible dig in the gravel with the draft coming from it. Mostly gravel so would need a lot of work""")

create_updates_and_images(cave_name="TR9", year="2014", images=[], description="""- Discovered in 2014 [HM3, page 63]
  - Obvious way on is choked, no indication of how easy digging / rock removal might be""")
create_updates_and_images(cave_name="TR9", year="2022", images=["TR9-2022.jpeg"], description="""- Revisited in 2023
  - Howling draft coming out of the cave
  - Vocal connection to a shake hole behind the main entrance
  - A dig was found at the back in a tight crawl and pushed for 5m and continues on for about another 5m
  - Very strong draft coming from the crawl, which turns to the right into the mountain""")

create_updates_and_images(cave_name="TR10", year="2023", images=["TR10-2023-1.jpeg", "TR10-2023-2.jpeg"], description="""- Logged as TR10 (no idea when)
- Relogged 2023
  - Unclear whether this was ever descended but a 5m hole which looks enticing
  - Not descended due to lack of gear
  - A TR11 GPS point was also logged but couldn't find any cave there within a 15m radius""")

create_updates_and_images(cave_name="U-Bend 571", year="2000", images=[], description="""- Another entrance to the main system that quickly connects with Primadona
- First mentioned in HM1, page 120, 2000""")
create_updates_and_images(cave_name="U-Bend 571", year="2007", images=[], description="""- Connected to Primadona in 2007 [HM2 draft, page 14]""")

create_updates_and_images(cave_name="Vilinksa Jama", year="2008", images=[], description="""- Discovered in 2008 [HM2 draft, page 43]
  - Connects to Vrtnarija""")

create_updates_and_images(cave_name="Vrtnarija, Gardener's World", year="1998", images=[], description="""Entrance into the main system that began as a dig by Ben Ogborne [HM1, page 99, 1998]""")
create_updates_and_images(cave_name="Vrtnarija, Gardener's World", year="2000", images=[], description="""The entrance dig broke into the passage in 2000 [HM1, page 128]""")

Update already exists with ID: 1
quality: 40
width, height: (1512, 2016)
Image already exists with ID: 1
Update already exists with ID: 2
Entry already exists for update_id: 2 and image_id: 1
Update already exists with ID: 3
quality: 50
width, height: (1512, 2016)
Image already exists with ID: 2
Update already exists with ID: 4
Entry already exists for update_id: 4 and image_id: 2
Update already exists with ID: 5
quality: 45
width, height: (1512, 2016)
Image already exists with ID: 3
Update already exists with ID: 6
Entry already exists for update_id: 6 and image_id: 3
Update already exists with ID: 7
quality: 55
width, height: (1512, 2016)
Image already exists with ID: 4
Update already exists with ID: 8
Entry already exists for update_id: 8 and image_id: 4
Update already exists with ID: 9
Update already exists with ID: 10
quality: 35
width, height: (1512, 2016)
Image already exists with ID: 5
Update already exists with ID: 11
Entry already exists for update_id: 11 and image_id: 5
qual

# Image sizing tests

In [80]:
insert_image_if_not_exists(db_path, "./images/A1-2022.jpeg")

quality: 40
width, height: (1512, 2016)
New image inserted with ID: 10


10

In [81]:
import base64

base64_string = """..."""

# Removing any non-base64 characters (newline, etc.)
base64_string = base64_string.replace("\n", "")

# Decoding the base64 string
try:
    image_data = base64.b64decode(base64_string)
    with open("output_image.jpg", "wb") as f:
        f.write(image_data)
    print("Image saved successfully.")
except Exception as e:
    print("An error occurred while decoding the base64 string:", e)


Image saved successfully.


# 

In [3]:
atlas_text = '''
### A1

2022:

<div>
  <img src="images/A1-2022.jpeg" width="20%"/>
</div>

- Discovered in 2008 [HM2 draft, page 44]
  - No information other than that they were logged and they are "worth a go down and look"
- Relogged in 2022
  - Rhys remembers going down one of them once upon a time so likely dead?
  - A1 Has two stainless bolts in it

### A2

2022:

<div>
  <img src="images/A2-2022.jpeg" width="20%"/>
</div>

- Discovered in 2008 [HM2 draft, page 44]
  - No information other than that they were logged and they are "worth a go down and look"
- Relogged in 2022
  - Rhys remembers going down one of them once upon a time so likely dead?

### A3

2022:

<div>
  <img src="images/A3-2022.jpeg" width="20%"/>
</div>

- Discovered in 2008 [HM2 draft, page 44]
  - No information other than that they were logged and they are "worth a go down and look"
- Relogged in 2022
  - Rhys remembers going down one of them once upon a time so likely dead?

### A4

2022:

<div>
  <img src="images/A4-2022.jpeg" width="20%"/>
</div>

- Discovered in 2008 [HM2 draft, page 44]
  - No information other than that they were logged and they are "worth a go down and look"
- Relogged in 2022
  - Rhys remembers going down one of them once upon a time so likely dead?

### ABBA 4

- Possible cave entrance found by BenR on the east face of planje during N9 trip in 2019
- ABBA 4 is likely to be 46.2676, 13.75168
- Should revisit to confirm

### ABBA 5

- Possible cave entrance found by BenR on the east face of planje during N9 trip in 2019
- ABBA 5 is likely to be 46.26749, 13.75123
- Should revisit to confirm

### Andy's Secret Dig

2022:

<img src="images/Andy's Dig-2022.jpeg" width="25%"/>

- No longer a secret as of 2022
  - Relogged on the GPS
  - Still diggable but pretty sketchy, with tent poles holding up boulders
  - Reached by going along the old Mig path before turning down a gulley
  - Can also be reached from below, walking up the gulley
  - From the path down the Gulley, climb up towards an obvious alcove
  - Dig is in the back left of the alcove, and down a 5m free climb

### B5

2022:

<div>
  <img src="images/B5-2022.jpeg" width="44.5%"/>
  <img src="images/B5 Close Up-2022.jpeg" width="25%"/>
</div>

- Spotted in 2022 from red paint marking
  - No known information
  - Gravel filled and would require a lot of work
  - Very mossy, not promising

### B10

2023:

<div>
  <img src="images/B10-2023-1.jpeg" width="44.5%"/>
  <img src="images/B10-2023-2.jpeg" width="25%"/>
</div>

2010:

<img src="images/B10-2010.jpg" width="25%"/>

First discovered during the blowing holes recce of 1995 [HM3, page 61, page 104]
- Photos show some surface bashing in 2010
- Dug in 2014
  - Contains a narrow tube suitable for digging
  - More digging required
- Relogged in 2023
  - Didn't go in, but looks interesting from the outside

### B12

2022:

<img src="images/B12-2022.jpeg" width="25%"/>

- Spotted in 2022 from red paint marking
  - No known historical information
  - Red paint can just be seen in bottom of image
  - Appears to be a 10m drop blocked by boulders
  - Digging attempted without tools but little progress made
  - Requires long chisel for entrance widening and oversuit to protect (squeeze not passed due to this)
  - Entrance may need traverse line for protection through squeeze
  - Definitely worth returning with tools

### B20

2022:

<img src="images/B20-2022.jpeg" width="25%"/>

- Re-logged in 2022
  - Good digging potential but hasn't been investigated to the end
  - If it connected with M2, it would give extra height to the system

### B91

2023:

<div>
  <img src="images/B91-2023-2.jpeg" width="44.5%"/>
  <img src="images/B91-2023-1.jpeg" width="25%"/>
</div>

- Relogged in 2023
  - Small hole 2m deep looks diggable, can’t see if completely dead. No obvious draft
  - Should rename this as B91 means nothing

### B93

2023:

<div>
  <img src="images/B93-2023.jpeg" width="50%"/>
</div>

- Relogged in 2023
  - Should be renamed as B93 doesn't mean anything
  - Small hole going sideways 2m, no obvious draft but easy digging
  - 
### B94

- No known information, near JJ2. Location 46°15'16"N 13°45'30"E. In archive "2019_Migoveccavelocations" GPX file.

### B95

- No known information, near bone cave. Location 46°15'15"N 13°45'33"E. In archive "2019_Migoveccavelocations" GPX file.

### B96

- No known information, near JB and JJ2, a bit down the cliff face. Location 46°15'16"N 13°45'28"E. In the archive "2019_Migoveccavelocations" GPX file.

### B97

- No known information. Might be the same as "JB" as it's logged very close. Just south of JJ2. Location 46°15'15"N 13°45'29"E. In archive "2019_Migoveccavelocations" GPX file.

### B98

- No known information. About 60m due west from B93. Location 46°15'13"N 13°45'31"E. In archive "2019_Migoveccavelocations" GPX file.

### B99

- No known information. About 60m WSW from B98, down the cliff face. Location 46°15'12"N 13°45'28"E. In archive "2019_Migoveccavelocations" GPX file.

### B100

- No known information. Was in one of the yellow club GPSs, but not in the archive "2019_Migoveccavelocations" file. Located in the cliff face NW of V10. Added to archive file as of 25/06/2024. Location 46.25597 N, 13.75801 E.

### Bone Cave

2022:

<img src="images/Bone-2022.jpeg" width="25%"/>

- Mentioned in passing in HM1, page 215, but no information provided other than it being near the V10 valley
- 2022
  - Note that Bone on the old GPS coordinates seems to correspond to a random shake hole by the tents, likely a different cave since V10 valley infers it being on the cliff
  - No cave there, only a 1 foot deep hole
  - Image is of the shake hole by the tents

### C1

2022:

<img src="images/C1-2022.jpeg" width="25%"/>

- Area C being the area around the container
- Spotted and logged in 2022
  - No previous entrance marked on the GPS in the area
  - Goes around 10m until a small boulder filled chamber

### C2, Bonsai Pot, Teeny Tiny Rhododendron Pot

2023:

<div>
  <img src="images/C2-2023-1.jpeg" width="44.5%"/>
  <img src="images/C2-2023-2.jpeg" width="44.5%"/>
</div>

- (Re)discovered in 2023
  - Named as there are loads of small Rhododendrons around the entrance
  - Large initial pit with various side chambers
  - Low crawl leads to various small chambers which all die, some after squeezes and short down climbs
  - Short climb up leads to a pitch found to have spits
  - This pitch descends quite a way, splitting into three routes all of which are blocked and die

### C3

2023:

<div>
  <img src="images/C3-2023-1.jpeg" width="44.5%"/>
  <img src="images/C3-2023-2.jpeg" width="44.5%"/>
</div>

- Spotted in 2023
  - Not yet descended
  - No signs of spits, but could have been dropped using naturals

### Coincidence Cave

- Discovered in 2015 [HM3, page 113]
  - Initially dug but requires more work
- Revisited in 2018 (not written up)
  - Breakthrough made to series of chambers
- Further explored by the Slovenians after 2018
  - All leads seem to have died, from word of mouth
- Revisited in 2022
  - End of cave terminates in boulder floor at base of pitch, this would be difficult to dig
  - According to Matti, draft disappears into hole above final pitch, may be diggable
  - Perry has started bolt climbing avens from entrance, first one dead

### D1, Desperation 1

2023:

<div>
  <img src="images/D1-2023-1.jpeg" width="44.5%"/>
  <img src="images/D1-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered by David Wilson Jnr
  - Named desperation due to convenient use as an emergency toilet
- Revisited in 2023
  - Doesn't seem to go anywhere, not promising

### DW's 1996 Dig

- Nearby to Royston Vasey, mentioned in HM2 draft page 44, 2008
  - Worth going back down again
- Relocation was attempted in 2022
  - No obvious dig could be found, may be the small filled in hole 5m towards Kuk from Royston Vasey
  - Not promising

### E1

2022:

<div>
<img src="images/E1-2022.jpeg" width="25%"/>
</div>

2008:

<div>
<img src="images/e1-2008.jpg" width="25%"/>
</div>

2007:

<div>
<img src="images/e1-2007.jpg" width="25%"/>
<img src="images/e1-2007(2).jpg" width="25%"/> 
</div>

- Discovered in 2007 [HM2 draft, page 14]
  - A 25m pitch leading to small windows which require widening to pass
- Revisited in 2008 [HM2 draft, page 41]
  - "small crawl" ongoing, but abandoned. Still a lead?
  - "deep deep" crawl not dead, dug out to very tight pitch-head. Surface survey, tied centerline to M2 entrance
- Revisited in 2009 [HM2 2019 draft, page 64]
   - cave approx 30m deep
   - dropped second pitch (capping and chiselling) into chamber, dug in floor to bedrock. Quote: "No better nor worst than capped pitch, but not the stunning lead
we were hoping for." Photos in Slov 2009 folder

### E2

2022:

<img src="images/E2-2022.jpeg" width="25%"/>

- Relogged in 2022
- Visited either 2018 or 2019
  - Ben R remembers poking around it with Janet at some point
  - No obvious leads that he can remember
  - Could be revisited

 
### E6

2007:

<img src="images/e6-2007.jpg" width="25%"/>" 

- Photo from 2007, information from filename. Red paint marking "E7" noted. No other info
- Unclear if a cave
- could be revisited


### E7

<img src="images/e7-2007.jpg" width="25%"/>" 

- Photo from 2007, information from filename. No other info
- Unclear if a cave
- could be revisited


### E8

2022:

<img src="images/E8-2022.jpeg" width="25%"/>

- Not a cave
  - Logged as "obvcave" on historical GPS files
  - Just an alcove, like K1
  - Small free climb through to the roof


### Gelateria, B92

2023:

<div>
  <img src="images/Gelateria-2023-1.jpeg" width="44.5%"/>
  <img src="images/Gelateria-2023-2.jpeg" width="44.5%"/>
</div>

- Logged as B92 on old GPS data (not sure when)
- Uncovered in 2019 due to low snow levels
  - Dug a little but not very promising
  - It is a real cave but get's very tight very quickly and the digging could continue for a long time
- Poked again in 2023
  - Christened Gelateria since it's next to where we get snow for the bivi
  - Still doesn't look promising, but recovered some digging things left from 2019

### Gondolin

- Discovered in 2017 [HM3, page 232]
  - Pushed for three pitches before the cave dies

### Hare Cave

2022:

<img src="images/Hare-2022.jpeg" width="44.5%"/>

- Description in HM1, page 183, 2004
  - Was guarded by a large hare in 2003, so left until 2004
  - Draughting strongly from the floor
  - Seems to be a false floor at the top of a rift, could be dug
  - Small phreatic "torpedo" tubes found in upper levels
- Revisited in 2005 [HM1, page 191]
  - Torpedo tubes all found to close down, no leads left
- Revisited in 2006 [HM1, page 206]
  - Investigated a phreatic tube but it didn't seem to go anywhere
  - A strongly draughting dig in the centre of the chamber but more digging is required to see if this goes or not
- Revisited in 2017 [HM3, page 231]
  - As previously reported, the "torpedo tubes" do not go and no more leads were found

### Jailbreak

- Discovered in 2013 [HM3, page 34]
  - Although it leads to a chamber with many ways on, all of them seem to die
  - Perhaps possible to dig, but no easy places
- Revisited and killed in 2014 [HM3, page 58, page 104]
  - Survey and description in HM3 reference
  - Main lead was killed after hauling a boulder out of the way, other lead was also killed

### Janet's Little Cave, MLC, My Little Cave

2022:

<img src="images/MLC-2022.jpeg" width="25%"/>

2009:

<div>
<img src="images/MLC-2009.jpg" width="44.5%"/>
<img src="images/MLC-2009(2).jpg" width="44.5%"/>
</div>


- Description in HM1, page 184, 2004
  - Small airy passage in the side of a shakehole near the Kuk path
  - Tube from first chamber into second chamber that hasn't been pushed
  - Ongoing lead as of 2004
- Explored and surveyed in 2019 by Tanguy Racine
  - This was when the Disto data was added to the GitHub repo
  - Presumably found to be dead? No indication of any leads
- Revisited by DPW and Leo Antwis 2022
  - Dug further but slow progress and not promising
  - Also found cave to be dead, after checking thoroughly for leads (including through tight hole)

### JCC

2023:

<div>
  <img src="images/JCC-2023-1.jpeg" width="44.5%"/>
  <img src="images/JCC-2023-2.jpeg" width="44.5%"/>
</div>

- Unsure if this is a cave or not, a GPS waypoint of this name is mentioned in HM2 draft, page 44, 2008
  - East of M19 there is large entrance with a draughting dig that looks promising
- Relogged in 2023
  - This description does indeed hold true, but dig wasn't investigated

### JJ1

- Discovered in 2006, HM1 page 214
  - Multiple caves in the V8 valley, unclear which is JJ1 from HM1 description alone
  - Seems to be a small chamber, a blowing aven and an unpushed wet crawl

### JJ2, Pella's Feathers

- Discovered in 2006, HM1 page 214
  - Many caves in the V9 valley, but this one is an obvious triangular hole
  - A crawl continues for 10m before splitting
  - The left branch dies, whilst the right branch ends with a rock that requires clearing to progress further

### JJ3, Hawk Cave

- Discovered in 2006, HM1 page 216
  - Initially discovered by Andrej Fratnik and "the other Andrej" back in the 70s
  - Rough survey and description given in HM1 (see above page)
  - Many drafting leads left due to lack of digging equipment and over suits
- Revisited in 2007 [HM2 draft, page 15]
  - Left ongoing due to lack of rope and time

### JJ4

- Discovered in 2006, HM1 page 215
  - A winder on the side of Kuk
  - No indication of any leads

### JJ5

- Discovered in 2006, HM1 page 215
  - Crawls and chambers end in a desperate dig beneath hanging death

### JJ6

- Not a cave, but a waypoint set in 2006, HM1 page 215
  - Many caves are visible from this point

### K1, The Goat Shelter

2022:

<img src="images/K1-2022.jpeg" width="25%"/>

- An obvious entrance on the flank of Kuk that only goes to a higher entrance with no leads [HM1, page 195, 2005]

### K2

2022:

<img src="images/K2-2022.jpeg" width="25%"/>

- Discovered in 2005 [HM1, page 195]
  - Easy digging and massive draught
  - Rock is very loose
  - Two days of digging in 2005
- Revisited in 2012 [HM2 draft, page 136]
  - Seems to have the most potential of all the area K caves
  - Much digging required before it goes
- Revisited in 2015 [HM3, page 140]
  - Found to "not go" although given it was an existing dig, this may have been subjective
- Revisited in 2022
  - Goes into small chamber with dig off to the side
  - Possible to dig in crawl sloping up towards the surface
  - No draft felt on a very warm and still day
  - Not very promising

### K3

- A large 30m shaft on top of Zeleni Vrh, believed bottomed in the early 90s [HM1, page 195, 2005]

### K4

- Entrance on north face of Kuk, found to just be a window with light filtering in from above [HM1, page 195, 2005]

### K5

- Entrance on west end of north Kuk face [HM1, page 195, 2005]
  - Abseiled down to scree floor but unclear if cave is completely dead

### K6

2022:

<img src="images/K6-2022.jpeg" width="25%"/>

- First mentioned 2005
  - Sharp crawl through to mud floor opening out with small draught, could be pushed further with elbow pads [HM1, page 195, 2005]
- Revisited in 2012 [HM2 draft, page 136]
  - Good progress was made with digging
- Revisited in 2022
  - Closes down into a dig which gets narrower not bigger
  - No obvious draft on a very warm and still day
  - Could be dug further but not very promising

### K7

2005:

<div>
  <img src="images/K7-2005-1.jpeg" width="44.5%"/>
  <img src="images/K7-2005-2.jpeg" width="44.5%"/>
</div>

- A dodgy chamber reached by a 5m clamber over boulders [HM1, page 195, 2005]
  - Small draught, may be possible to dig

### K8

- Overhang into a small choked chamber [HM1, page 195, 2005]

### K9

2022:

<img src="images/K9-2022.jpeg" width="25%"/>

- Quickly opens out into a 15m pitch [HM1, page 196, 2005]
  - A twin pitch is reachable from a large window
  - Dig potential in a draughting collection of rubble
- Revisited in 2022
  - Located the entrance to photograph and log the GPS location, but pitch not descended

### K10, Torn Testicle

2022:

<img src="images/K10-2022.jpeg" width="25%"/>

- A very small hole currently blocked but with obvious continuation [HM1, page 196, 2005]
  - Currently only descends 3m
  - stoned rattle when pushed to the north
  - Would need to remove sharp rock blades and lift out boulders in order to progress any further
- Revisited in 2022
  - Human sized hole in pavement that's very grabby and unpleasant
  - Person sized alcove at the bottom, with another person sized alcove off to the side at floor level
  - No indication of any continuation or possible digs, not promising

### K11

2022:

<img src="images/K11-2022.jpeg" width="25%"/>

- Scree climb into a chamber at the end of a shakehole with a cold draught and good digging possibilities [HM1, page 196, 2005]
- Revisited in 2012, presumed to be reasonably promising but not incredible [HM2 draft, page 136]
- Revisited in 2022
  - Obvious entrance in large shake hole which leads to moderately sized chamber
  - Small descending hole on right side of entrance which may be possible to dig

### K12, Victoria Coach Station

2022:

<img src="images/K12-2022.jpg" width="25%"/>

- A very large entrance leading to a large snow slope [HM1, page 196, 2005]
  - Enormous draught in places, but many crawls blocked with ice and rubble
- Revisited in 2012, presumed to be reasonably promising but not incredible [HM2 draft, page 136]
- Revisited in 2015 [HM3, page 140]
  - Two days of digging resulted in slow progress
  - Crawl get's narrower
  - Still a "promising" lead
- Revisited in 2022
  - Lead to back of main chamber is a dig continuing in a few meters but with a very loose roof which rocks fell from when crawled through
  - Dig is ongoing but sketchy given looseness of surrounding rock
  - Small alcove found between boulders and snow plug by entrance, no continuation but with more melting this may reveal further alcoves
  - No other leads / crawls could be found, leaving only the scary dig

### K13, Pedestrian Subway / Arrivals Hall

2022:

<img src="images/K13-2022.jpeg" width="44.5%"/>

- Rift directly above K12 with possible dig to the South East and rift in floor connecting with the K12 main chamber [HM1, page 197, 2005]
- Revisited in 2022
  - An infuriatingly good entrance that is completely free of rock and gravel, but unfortunately goes straight back into the roof of K12
  - No indication of any other leads, not promising

### K14

2022:

<img src="images/K14-2022.jpeg" width="25%"/>

- A shallow rift leading to a small chamber, likely to break onto the surface but not pushed fully [HM1, page 197, 2005]
- Revisited in 2022
  - Rift drops down into boulder filled chamber
  - Route going towards surface
  - Another possible dig underneath the entrance but not easy
  - Overall not promising

### K15, K15i (The Rik-Venn Imaginary Series)

2022:

<img src="images/K15-2022.jpeg" width="25%"/>

- A big blowing and cold cave ending in a dig [HM1, page 197, 2005]
  - Goes in ten meters south and down before reaching a choke underneath a suspended boulder
  - Choke has been dug for one hour but lacked crowbar
- Revisited in 2022
  - Good entrance which goes into medium sized chamber but quickly chokes
  - Many small birds living in the cave at time of visit

### K16, The Escalator

2022:

<img src="images/K16-2022.jpeg" width="50%"/>

- A series of holes by the Skrbina path [HM1, page 197, 2005]
  - Many alcoves, strong draught and ice plug at bottom
- Revisited in 2022
  - Can be approached from any direction, but be very careful when approaching from above as the entrance is hidden beneath a small overhang making easy to fall into given the steep scree slope
  - A series of large entrances to a single large chamber with steep scree slope
  - At the top of the slope, a small chamber which is not hard to reach was not investigated as we ran out of time
  - At the bottom of the slope there is a small entrance to another chamber, however the entrance is very small and there is a risk of it being sealed by falling scree upon entry
  - Promising, should be revisited

### K17

- Small chamber at top of shakehole on Podriagora (spelling may be incorrect in HM1?) [HM1, page 197, 2005]
  - Easy digging at end but loose and scary

### K18

- Rift near K17 disappears into a 5m pitch to the floor, not bottomed due to lack of rope and hanging death [HM1, page 197, 2005]

### K19, Coach-Crash Dig

2005:

<div>
  <img src="images/K19-2005-1.jpeg" width="44.5%"/>
  <img src="images/K19-2005-2.jpeg" width="44.5%"/>
</div>

2022:

<img src="images/K19-2022.jpeg" width="25%"/>

- Entrance rift in a pile of boulders drops down a few meters the runs down into a gravel pile and opens out [HM1, page 198, 2005]
  - Infamous near death experience for Dave Wilson
  - Still possible to access the cave underneath the fallen boulder
  - Boulder now wedged in rift requires splitting, preferably with caps
- Revisited in 2012 [HM2 draft, page 136]
  - Digging continued but no further information
- Revisited in 2022
  - Dig looks to have been filled in by falling rock and scree given it's exposed location at the bottom of a large scree slope
  - Gravel chamber is still visible 2 m down, but any digging would result in the surrounding rock collapsing inwards
  - Any dig likely to be filled in within a year or two or scree and rock fall
  - Not promising

### K20, K-Moss

2022:

<img src="images/K20-2022.jpeg" width="25%"/>

- Near K15, a small mossy hole with a slight draught [HM1, page 198, 2005]
  - Some digging carried out, but would be easy to continue further
- Revisited in 2022
  - Hardly an entrance, just a small mossy depression in the ground with no draught or cold air, on a very warm and still day

### K21

- First mentioned as being revisited in 2012 [HM2 draft, page 136]
  - No further information, but not included in list of "not promising" entrances, so presumed to be reasonably promising

### K22

- First mentioned as being revisited in 2012 [HM2 draft, page 136]
  - "Not at all promising"

### K23

- First mentioned as being revisited in 2012 [HM2 draft, page 136]
  - No further information, but not included in list of "not promising" entrances, so presumed to be reasonably promising

### K24

- First mentioned as being revisited in 2012 [HM2 draft, page 136]
  - "Not at all promising"

### K25

- First mentioned as being revisited in 2012 [HM2 draft, page 136]
  - "Not at all promising"

### K26

- First mentioned as being revisited in 2012 [HM2 draft, page 136]
  - No further information, but not included in list of "not promising" entrances, so presumed to be reasonably promising

### K27

2022:

<img src="images/K27-2022.jpeg" width="25%"/>

- First logged in 2022
  - Logged after thinking it was K14
  - Short drop into gravel filled chamber
  - No continuation or possible digs, not promising

### K28

2022:

<img src="images/K28-2022.jpeg" width="25%"/>

- First logged in 2022
  - A fairly small, draughting, rubble-filled hole about a meter cubed
  - Likely to have been snow covered previously given how sheltered and cold it is
  - Considering draught was felt on a very warm still day, this is a promising indication of nearby cave passage
  - No continuation or digging potential, not promising as a lead in itself

### K29

2022:

<img src="images/K29-2022.jpeg" width="25%"/>

- Previously logged as K99 on the old GPS files, but no mention of it in any hollow mountain editions so unclear why it broke with the area K naming convention
- Logged as K29 in 2022
  - A small chamber 5m underneath large boulders
  - Leads to a second small alcove
  - Not promising as a lead in itself, but cold
  - No obvious draught on a very warm and still day

### Keyhole Cave

- Very close to Postcard Cave, see Postcard cave entry for description of location

### LP1

2022:

<img src="images/LP1-2022.jpeg" width="25%"/>

- Discovered in 2022
  - LP = Limestone Pavement, as previously caves here were named TR1,2,3 etc. but this system breaks down when anyone other than Tanguy finds a cave
  - Small chamber filled with rocks which can be reached from multiple entrances
  - Additional entrance has obvious cave below, likely providing a better entrance but blocked by large rocks
  - Initial hauling and digging of this better entrance
  - Inside loose slope leads to snow plug with crawl going around corner beyond, not pushed as no oversuit available
  - No breakthrough made - good potential, worth revisiting

### LP2

2022:

<img src="images/LP2-2022.jpeg" width="25%"/>

- Discovered in 2022
  - 2m descent to snow plug
  - Draught disappears in front and to the left in a small rift not possible to dig
  - Snow tasted of high quality, according to Leo
- Revisited in 2023 by Julien
  - Small tunnel to left which rocks bounce down for a while
  - Potentially leads to pitch

### LP3

2022:

<img src="images/LP3-2022.jpg" width="25%"/>

- Discovered in 2022
  - Not descended, no notes taken at the time
  - May be worth revisiting
- Killed in 2023
  - Bolted and descended about 10m down on single bolt backed up with natural
  - Snow plug about 5m down with pitch continuing to the side of it
  - Passage becomes choked and grabby
  - Slight draft but small and too difficult to dig, effectively dead

### LP4

2022:

<img src="images/LP4-2022.jpeg" width="25%"/>

- Discovered 2022
  - May have been descended before
  - Likely plugged by snow
  - Unclear from notes whether this is worth returning to
- Visited 2023
  - Didn't go down but stone rattles down a long way
  - Quite promising

### LP5

2022:

<img src="images/LP5-2022.jpeg" width="25%"/>

- Discovered 2022
  - Big hole with gravel and snow at the bottom
  - Likely connects with LP4
  - Unclear from notes whether this is worth returning to

### LP6

2022:

<img src="images/LP6-2022.jpeg" width="44.5%"/>

- Discovered in 2022
  - Hole in side of shake hole blocked by boulder with visible slope underneath
  - Looks possible to widen the entrance
  - Worth revisiting
  - Also logged as TJ1

### LP7, Gardeners of the Galaxy, The Jaws of Death

2022:

<img src="images/LP7-2022.jpeg" width="25%"/>

- Discovered in 2022
  - A new hole had opened up in a shake hole since the previous expedition before covid
  - Far too loose to descend this year, so instead we chucked a load of rocks down it (many good videos)
  - These are the jaws of death that nearly ate Rory Rose

### LP8

2022:

<img src="images/LP8-2022.jpeg" width="25%"/>

- Discovered in 2022
  - Not descended but look interesting given entrance is recently exposed above snow plug
  - Worth revisiting, since new entrances may have been revealed
- Revisited in 2023
  - A snow crawl was found but it didn't go very far
  - Rest of the cave seems dead, but high snow levels

### LP9

2022:

<img src="images/LP9-2022.jpeg" width="25%"/>

- Discovered in 2022
  - Only discovered this cave after leaving the mountain
  - Had initially thought this was PF10 but apparently not, since actual entrance was instead logged as M24, and this entrance as PF10
  - The "M24" entrance, which is larger and much further down the pavement has been confirmed as PF10
  - Rocks fall a fair way, definitely worth a look
  - Next to and just above Alex Seaton's old dig into the side of a cliff
- Revisited in 2023
  - Entrance is full of snow so can't hear rocks fall down this year
  - Neither Arun or Astrid could fit into the cave, as entrance is much smaller than it looks
  - Would need some widening for anyone to investigate further

### LP10

2023:

<div>
  <img src="images/LP10-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP10-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered 2023
  - Draughting slightly outwards
  - Could be dug but would require more effort than it's worth
  - Very cold

### LP11

2023:

<div>
  <img src="images/LP11-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP11-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Draughting out slightly
  - Tight rift continues down for about 4m but not investigated due to lack of over suits

### LP12

2023:

<div>
  <img src="images/LP12-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP12-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered 2023
  - Opens out to a 3m x 1m chamber at the bottom
  - Possibly possible to dig, but very choked

### LP13

2023:

<div>
  <img src="images/LP13-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP13-2023-2.jpeg" width="44.5%"/>
</div>

- Logged in 2023
  - Can see a continuation past the snow at the bottom of a 5m shake hole
  - Not descended due to lack of gear

### LP14

2023:

<div>
  <img src="images/LP14-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP14-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered and killed in 2023
  - Reasonably sized shake hole but dies quickly
  - Free climbed down
  - Dead, completely choked with gravel

### LP15

2023:

<div>
  <img src="images/LP15-2023-1.jpeg" width="30%"/>
  <img src="images/LP15-2023-2.jpeg" width="30%"/>
  <img src="images/LP15-2023-3.jpeg" width="30%"/>
</div>

- Discovered and killed in 2023
  - Right next to the jaws of death
  - Initially very promising but found to link to next shake hole after 10m
  - Extensive digging allows for a cracking through trip
  - No obvious continuation, killed

### LP16, Icehorn

2023:

<img src="images/LP16-2023.jpeg" width="50%"/>

- Discovered in 2023
  - Squeezing past the snow plug leads to a large chamber 5m
  - Many small holes leading off of the main shake hole which all seem to drop into the main chamber when throwing rocks down
  - Dig at bottom of large chamber leads to an ongoing crawl but dies quickly in all directions after being dug
  - Another dig up in the left wall of the chamber dies quickly
  - All other leads seem to die

### LP17

2023:

<div>
  <img src="images/LP17-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP17-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered and killed in 2023
  - Twin holes, climb down one and then stoop into the other
  - Looked promising and descended down for about 5m but dies
  - No obvious digs or drafts

### LP18

2023:

<div>
  <img src="images/LP18-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP18-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Not descended but possible continuation out of sight
  - Not that promising but worth a look

### LP19

2023:

<div>
  <img src="images/LP19-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP19-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - 7m hole with possible continuation out of sight
  - Not descended as looked at other holes instead

### LP20

2023:

<div>
  <img src="images/LP20-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP20-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Previously logged as TJRIFT
  - Large 10m deep rift definitely worth a look
  - Two entrances and another two holes just down the valley which may also be connected

### LP21, Tin Can Hole

2023:

<img src="images/LP21-2023.jpeg" width="50%"/>
  
- Discovered and killed in 2023
  - Dropped previously since we found a fish tin at the bottom
  - No obvious digs all completely full of rubble 
  - Dropped on a rope but may have been descended by crazy free climb before, no spits seen

### LP22

2023:

<img src="images/LP22-2023.jpeg" width="50%"/>
  
- Discovered in 2023
  - Small dark not drafting hole
  - Not investigated since no over suit but worth a look

### LP23

2023:

<img src="images/LP23-2023.jpeg" width="50%"/>
  
- Discovered and killed in 2023
  - Can walk in
  - Dead, completely choked with gravel

### LP24, (Arun's) Secret Hole, How much is that Chamois in the window

2023:

<div>
  <img src="images/LP24-2023-1.jpeg" width="30%"/>
  <img src="images/LP24-2023-2.jpeg" width="30%"/>
  <img src="images/LP24-2023-3.jpeg" width="30%"/>
</div>

- Discovered and killed in 2023
  - Dropped by Astrid and Arun
  - Incredibly tight and sharp, required much bashing and multiple bolts
  - Dead chamois found on one ledge halfway down
  - Cave chokes up with gravel, no obvious digs or drafts
  - An impressive ice stal at the bottom

### LP25

2023:

<div>
  <img src="images/LP25-2023.jpeg" width="50%"/>
</div>

- Discovered in 2023
  - Not investigated since focussing on other caves
  - Very close to JOD
  - Worth a free climb down, quick to check

### LP26

2023:

<div>
  <img src="images/LP26-2023.jpeg" width="50%"/>
</div>

- Discovered in 2023
  - Not investigated since focussing on other caves
  - Very close to JOD and next to LP25
  - Worth a free climb down, quick to check

### LP27

2023:

<div>
  <img src="images/LP27-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP27-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Snow plugged rift not descended
  - Doesn't look that promising

### LP28, Christmas Pot

2023:

<div>
  <img src="images/LP28-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP28-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Right next to the LP Christmas tree
  - Dies in boulders at the bottom
  - Possible dig but large overlying boulders mean probably not worthwhile

### LP29

2023:

<div>
  <img src="images/LP29-2023.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Not descended but looks like a possible dig at the bottom
  - Likely free climbable

### LP30

2023:

<div>
  <img src="images/LP30-2023.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Not descended but looks like a possible dig at the bottom
  - Likely free climbable

### LP31

2023:

<div>
  <img src="images/LP31-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP31-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Not descended but small chamber at the bottom with snow
  - Needs bolting

### LP32

2023:

<div>
  <img src="images/LP32-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP32-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Not descended but small chamber beneath a boulder

### LP33

2023:

<div>
  <img src="images/LP33-2023.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Not descended possible dig at the bottom
  - Needs bolting

### LP34

2023:

<div>
  <img src="images/LP34-2023.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Not descended since needs bolting
  - Large hole with ice at the bottom

### LP35

2023:

<div>
  <img src="images/LP35-2023.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Screen covered floor 4m down
  - Not descended

### LP36

2023:

<div>
  <img src="images/LP36-2023.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Pot with stream-way like walls
  - Bottom is filled with scree, 3-4m deep
  - Undescended

### LP37, Stalagm-ice

2023:

<div>
  <img src="images/LP37-2023-1.jpeg" width="44.5%"/>
  <img src="images/LP37-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered and killed in 2023
  - Obvious opening drops down to choked shaft
  - Small but impressive ice formations at back of chamber
  - No obvious digs or ways on

### M1

2022:

<img src="images/M1-2022.jpeg" width="50%"/>

- Depth in 1983 recorded as -61m [HM1, page 13]
- Revisited in 2007 [HM2 draft, page 13]
  - Repushed and resurveyed
  - One ongoing lead left in a window, which may require some chemical persuasion

### M2

2022:

<img src="images/M2-2022.jpeg" width="25%"/>

- An entrance to the system. Discovered by the Slovenes back in the 70s [HM1, page 6]

### M3

- No known coordinates, altitude or depth as of 1993 [HM1, page 13]
- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)

### M4

2022:

<img src="images/M4-2022.jpeg" width="50%"/>

- No known depth as of 1993 [HM1, page 13]
- Relogged in 2022
  - Not descended
  - M4 painted in blue, rather than red paint for some reason
  - May be worth revisiting, since close to the bivi

### M5

- No known coordinates, altitude or depth as of 1993 [HM1, page 13]
- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)

### M6

2007:

<img src="images/M6-2007.jpg" width="25%"/>


- Depth indicated as -67 m as of 1993 [HM1, page 13]
- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)
- Revisited 2007 [HM2 draft, page 13]
  - small extensions - "pretty little bit of stream formed new cave, ended in draughting bedding plane dig."
  - images of surface shaft and Tetley putting in a rebelay
  - resurveyed

### M7

2022:

<img src="images/M7-2022.jpeg" width="25%"/>

- Depth indicated as -20m as of 1993 [HM1, page 13]
- Relogged in 2022

### M8

2022:

<img src="images/M8-2022.jpeg" width="25%"/>

- Depth indicated as -13m as of 1993 [HM1, page 13]
- Relogged in 2022

### M9

2022:

<img src="images/M9-2022.jpeg" width="50%"/>

- Depth indicated as -19m as of 1993 [HM1, page 13]
- Relogged in 2022

### M10

2022:

<img src="images/M10-2022.jpeg" width="25%"/>

- Depth indicated as -33m as of 1993 [HM1, page 13]
- Visited frequently to collect snow for water
- Explored more thoroughly in 2015 [HM3, page 111]
  - One way on dies in a too-tight rift
  - Another route dies in an icy section

### M11

- Depth indicated as -6m as of 1993.
- "It is difficult to understand why holes such as M11 (6m deep!) were noted, as the plateau has many similar or deeper surface holes which were not then formally recognised." [HM1, page 13]
- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)

### M12 maybe?

2022:

<img src="images/M12 maybe-2022.jpeg" width="25%"/>

- Depth indicated as -15m as of 1993 [HM1, page 13]
- Possibly relocated 2022
  - Not descended but found a cave which looks dug out, and is in the vicinity of M12 on the GPS
  - At the exact location of M12 on the GPS, there are no obvious entrances
  - M12 could be off the cliff, since some old coordinates are dozens of meters away from their actual location
  - Should definitely be investigated, can hear a small chamber when rocks thrown

### M13

- Depth indicated as -15m as of 1993 [HM1, page 13]
- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)

### M14

- Depth indicated as -22m as of 1993 [HM1, page 13]
- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)

### M15

2022:

<img src="images/M15-2022.jpeg" width="25%"/>

- Depth indicated as -58m as of 1993 [HM1, page 13]
- Relogged 2022

### M16

2022:

<img src="images/M16-2022.jpeg" width="25%"/>

- An entrance to the main system discovered by the Slovenes in 1982. [HM1, page 8]

### M17

2018:

<img src="images/M17-2018.jpg" width="25%"/>


- Depth indicated as -58m as of 1993 [HM1, page 13]
- Further explored in 1994, HM1 page 25 with survey
  - Pushed until it got too cold, going lead at bottom of cave
- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)

### M18, Torn T-Shirt Pot, Jama Strgane Srajce

2022:

<img src="images/M18-2022.jpeg" width="25%"/>

- Large entry about discovery in HM1 page 19 from 1994
- An entrance into the main system

### M19, BS Pot

- Entry about discovery in HM1 page 25 from 1994
  - Extremely tight and loose
  - Next to M17
  - Incredibly strong draft
  - Pushing stopped due to incredibly loose walls requiring shoring
- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)

### M20, White Shiver Pot

- Large entry about discovery and survey in HM1, page 17, 1994
  - Large 15m pothole with 20m drop to a snow plug
  - Squeezing past the snow plug led to a large chamber
  - No indication of continuation of digging potential
- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)

### M21, B9, Jackie's Blower

- Entry about discovery and survey in HM1, page 33, 1995
  - Two entrances near the top of the cliff
  - Very loose
- Revisited in 2016 [HM3, page 219]
  - Has apparently been visited a number of times since initial discovery
  - The bottom pitch was pushed but no mention of whether this lead is ongoing or not

### M22, Venus Cave

2022:

<img src="images/M22-2022.jpeg" width="50%"/>

- Entrance about discovery and survey in HM1, page 34, 1995
  - Named as it sucked flies in like a venus fly trap, strong draft!
- Explored in 2018 as seen in 2018 documentary video, no encouraging ways on after digging

### M23, Julie's Pantry Cave

- Entry on discovery and survey in HM1, page 32 from 1995
  - Very strong draft
  - Terminates at a small passage
- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)

### M24, PF10

2022:

<img src="images/M24-2022.jpeg" width="25%"/>

- Entry about discovery and survey in HM1, page 36, 1995
  - Crawl at bottom deemed too unstable to continue exploration
  - Abandoned for other leads in 1995
  - By the looks of EE, still a window on P4 and a bolt climb at bottom of P4 left unexplored. Worth a look. Could break into southern extensions.
- Relocated in 2022
  - Previously M24 and PF10 were logged as being over 100m apart
  - Upon throwing rocks down, sounds of ice smashing could be heard
 
<img src="images/PF10Survey.png" width="25%"/>

### M25, Gulliver's Kipper Cave

- Entry about discovery in HM1, page 33, 1995
  - On the face of Mig, visible from the shepherd's huts
  - Initially opens up but all proceeding chambers closed down
  - Little effort put into the cave, no survey taken. Could be worth a revisit for the katasterjam.
- Rediscovered by Ben R and Ben H in 2019. It is located up the coincidence valley and is a very big entrance with a P10 to a pile of wood. Was bolted in 1996 and is more or less dead (see HM1)
- Coordinates converted from GK5 to Lat/Long: 46.245408, 13.765006
- May have digging potential

### Monatip

- An entrance to the main system discovered in 2007 [HM2 draft, page 18]

### Moth Cave

2022:

<img src="images/Moth-2022.jpeg" width="50%"/>

2007

<img src="images/Moth Cave-2007.jpg" width="44.5%"/>

- Discovered in 2006 [HM1, page 207]
  - Located close to Hare cave
  - Left at a pile of draughting boulders which are easy and quick to dig
  - About an hour of digging was carried out in 2006
- Revisited in 2007 [HM2 draft, page 15, page 25]
  - Large volumes of gravel removed
  - All leads seem to have been killed
  - Only way on is another squeeze which would require rock removal to pass

### Mugo

2020:

<div>
<img src="images/Mugo Upper Entrance-2020.jpg" width="25%"/>
<img src="images/Mugo Lower Entrance-2020.jpg" width="25%"/>
</div>

- Discovered in 2020 [Slovenia subsite, 2020 Summer Action]
  - accessed from the contour path linking Kal to the head of the Tolminka valley
  - found while looking at overhangs and possible cave entrances that had been identified from below before ascending scree slopes below Gondolin
  - short descent unveiled a chamber which continued with a small climb into a dead end
  - two entrances
  - surveyed; final length 38m, depth 7m

### N01

2023:

<div>
  <img src="images/N01-2023-1.jpeg" width="30%"/>
  <img src="images/N01-2023-2.jpeg" width="30%"/>
  <img src="images/N01-2023-3.jpeg" width="30%"/>
</div>

- Not the same as the N1 in area N
- Rediscovered in 2014 [HM3, page 104]
  - Found to be 3m deep and leading into a small chamber but no sign of downward progression
- Relogged in 2023
  - Not dropped, red spray paint and gravel floor visible

### N1

- No known information
- Note that this is different to the N01 found to the south of Kuk

### N2

- No known information

### N3

- No known information

### N4

- No known information

### N5

- No known information

### N6

- No known information

### N7

- No known information

### N7

- No known information

### N8

- No known information

### N9, Kuk Pot

2012:

<img src="images/N9, Kuk Pot-2012.jpg" width="25%"/>


- First mentioned in 2012 [HM2 draft, page 136]
  - First two pitches dropped
  - Continuation visible but left due to lack of time
- Returned to and killed in 2019. Writeup on UKCaving [here](https://ukcaving.com/board/index.php?threads/maraton-iccc-jspdt-slovenia-expedition-july-2019.25199/).

### N10

- No known information

### N11

- Someone (unknown) recently logged this on one of the club GPS's. Coords are: 46°16'02"N 13°45'38"E
- Worth having a look

### N12

- Someone (unknown) recently logged this on one of the club GPS's. Coords are: 46°15'47"N 13°45'33"E

### N13

- Also logged as "BOLTCAVE" and "WB1". A vertical shaft with spits. No information on who originally descended this, no mentions in hollow mountains. Definitely worth checking out. Coords are: 46°15'47"N 13°45'37"E

### N14

- Also logged as "WB2". An approximately 5m deep vertical rift with possible cave at the bottom. Remains undescended because of lack of resources. Definitely worth checking out. Coords are: 46°15'44"N 13°45'38"E

### 0131

- Possibly an accidental GPS marker, so might not be a cave. Between N11 and N12. Coords are: 46°15'54"N 13°45'38"E

### P01 (North of Kuk)

- Included on the area N map, HM3 page 264
- No information regarding this cave was found

### P1

2023:

<div>
  <img src="images/P1-2023-1.jpeg" width="44.5%"/>
  <img src="images/P1-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Although very likely to have been discovered earlier, but no records of it
  - Very close to M17, M19, JCC
  - P series for Plateau as all the other caves in this area are either M series, which we don't continue any more, or have random names
  - Short 5m drop to a small chamber, can’t see the bottom with bright sunlight, don’t look easily climbable
  - Not descended due to lack of gear and time

### P2

2023:

<img src="images/P2-2023.jpeg" width="25%"/>

- Discovered in 2023
  - Again, likely looked in before
  - Can walk in but I’d tear my clothes, small chamber can’t see the back. Many fake caves around

### P3

2023:

<img src="images/P3-2023.jpeg" width="50%"/>

- Discovered in 2023
  - Again, likely looked in before
  - A bunch of holes above M17 which you’d need to be clipped in to investigate, but look as though they go back a fair way
  - Potentially one of these is Mouse cave but poorly logged? The GPS for it was a fair way off but didn't correspond to any obvious entrance

### P4

2023:

<div>
  <img src="images/P4-2023-1.jpeg" width="44.5%"/>
  <img src="images/P4-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Didn't drop in since no kit
  - Looks like it probably dies but worth a proper look
  - Small chamber may continue round to the left

### P5

2023:

<div>
  <img src="images/P5-2023-1.jpeg" width="44.5%"/>
  <img src="images/P5-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered in 2023
  - Didn't look inside since no kit but so close to cliff abseil that it should be looked at
  - Might be a good dig if nothing else

### Planika

2007:

<div>
  <img src="images/Planika-2007(1).jpg" width="44.5%"/>
  <img src="images/Planika-2007(2).jpg" width="44.5%"/>
</div>

- Discovered in 2007 [HM2 draft, page 18]
  - Large snow and ice filled cave with holes driven through it by water and air
  - Ends in an un-pushed ice tube after a scary boulder funnel
- Revisited in 2008 [HM2 draft, page 31]
  - No further progress on the route from the previous year made due to high snow levels
  - A new rift (implied name Echo Rift) found from a water drilled hole through the snow plug which was pushed down a pitch
  - Another chamber was found from a window
  - Upper rift "Diggers of the UG"

### Postcard Cave

- Briefly mentioned in HM1, page 183, 2004
  - In an adjacent shakehole to the east of Hare Cave
  - No further information
- Description from 2007 on club wiki along with keyhole cave:
  - "Keyhole cave and postcard cave are very near the bivvy. The easiest way to describe them is to start from the edge of the shakehole with hare cave in. Facing towards primadoni direction hare cave is on the left side of the shake hole. Basically you are standing above postcard cave. It is in a smaller shakhole behind you. The entrance is a very low thin bedding plane that immediately breaks into an 'impressive' entrance phreatic tube with a choked draughting rift in the floor. This looks like many other entrances, so the way to confirm is that keyhole cave should also be present in the shakehole. Still facing the same way (i.e. away from the shakehole towards the hare cave shakehole), keyhole cave is on the left side and is almost completely invisible unless you are standing in exactly the right place. Entrance is a lovely phreatic keyhole about 1m high and really very hidden. The cave follows this shape for a couple of metres before ending in a choked small chamber. Not that interesting but a great entrance - would make a superb start to a trip"

### Primadona

- A main entrance to the main system known about for a long time by the Slovenes
- First mentioned in HM1, page 118, 2000

### Pueblo

2022:

<img src="images/Pueblo-2022.jpeg" width="25%"/>

- Not a cave
  - Fratnik's old bivi spot from many decades ago
  - Still filled with metal tent poles
- Relogged in 2022

### Razor Cave

- A cave near the Razor hut, extensively explored by the JSPDT [HM1, page 208, 2006]
- Surveyed in 2007 [HM2 draft, page 14]

### Royston Vasey

2022:

<img src="images/Royston_Vasey-2022.jpg" width="44.5%"/>

- Entry about discovery and a survey in HM1, page 147, 2001
  - Very close to the bivi, in an 8m deep shake hole
  - Good digging potential given it's proximity to the bivi
- Rediscovered in 2008 [HM2 draft, page 44]
  - Not pushed any further, only rediscovered
- Re-bolted and dug in 2022
  - All possible leads investigated
  - Can hear cave continues onto another chamber at bottom of final pitch but not worth blasting since walls close in
  - Conclusively dead this time

### S1, East Pole

2009:

<img src="images/S1-2009.jpg" width="44.5%"/>


2007:

<img src="images/S1-2007.jpg" width="25%"/>


- Mentioned in HM1, page 99, 1998
  - Multiple chambers and small pitches, but very unstable
  - Multiple digging trips have since returned to the cave with no major breakthroughs
  - "[a] promising lead"
- Revisisted in 2007, 2008 and 2009
  - No write ups of the trips; only references to progress from tweet [Votla Gora tweets, 2:32 PM Aug 10th, 2008] given below
  - "S1: way to right hand pitch found by hammering rift above big pitch.Strong draft from tight meander, several hrs hard hammer.4th system?"
  - entrance photos and walk labelled under Eastpole
- Revisited in 2017 [HM3, page 271] [2017 logbook]
  - Did not manage to make progress on the upwards dig
  - Marginal progress was made over a tight rift, despite the loss of the lump hammer


### S2

2007:

<div>
  <img src="images/S2-2007.jpg" width="44.5%"/>
</div>

- Photo of entrance by Jarvist
- No known information

### S3

2007:

<img src="images/S3-2007.jpg" width="44.5%"/>

- Photo of entrance by Jarvist
- No known information

### S4

- No known information

### S5

2007:

<img src="images/S5-2007.jpg" width="44.5%"/>

- Discovered in 2013 [HM3, page 37]
  - Initially a free-climbable p6, followed by a p20 that requires a rope
  - Appears that this hasn't been descended
  - Confused whether the 2007 image is the same cave as Gergely's description from 2013 HM3

### S6

- Discovered in 2013 [HM3, page 37]
  - Strongly drafting, just as much as S1
  - Requires work but is the most promising lead in the area

### S7

<div>
  <img src="images/S7-2007.jpg" width="44.5%"/>
  <img src="images/S7-2007(pitch).jpg" width="44.5%"/>
</div>

- Discovered in 2013 [HM3, page 37]
  - Cold air drafting from between two boulders
  - Not an actual entrance, but could be dug into one
  - Photo of entrance by Jarvist
  - Confused whether the lablled 2007 images are the same cave as Gergely's description from 2013 HM3 given that the 2007 photos show a pitch. It's not even that clear whether they relate to the same cave as each other, they look a little different
 
### S8

2007:

<img src="images/S8-2007.jpg" width="25%"/>

- Photo of entrance by Jarvist
- No known information


### S12

2022:

<img src="images/S12-2022.jpeg" width="25%"/>

- Relogged in 2022
  - No known information
  - If this is S12, where are S8-11?

### S-A

2022:

<img src="images/S-A-2022.jpeg" width="25%"/>

- Relogged in 2022
  - No idea what this is, but there's a large shake hole there
  - Likely a nearby cave, but no obvious what the GPS point was referring to
  - Should ask some old(er) lags

### Samo1

2022:

<img src="images/SAMO1-2022.jpeg" width="25%"/>

- Unclear when Samo first logged this
- Revisited in 2022
  - Crawl goes down at least 5m to capped entrance to small chamber
  - Large draft and nearby to new Gardeners of the Galaxy hole, and clearly in the same line of shake holes as it, but cave heads in different direction
  - Not worth revisiting

### Samo2

2022:

<img src="images/SAMO2-2022.jpeg" width="25%"/>

- Unlcear when Samo first logged this
- Revisited in 2022
  - 10m deep shaft, gravel floor with ice plug 2 by 10 m diameter at bottom
  - Big draught and adjacent holes connects back in
  - Cave conclusively killed

### Stag Cave

- Discovered in 2007 [HM2 draft page 15]
  - Within 20m of the tents
  - An initial pitch leads to a dead chamber

### Storm cave

- Description in HM1, page 184, 2004
  - Main way on blocked by snow
  - Other entrances in the area which were blocked by snow

### Sunset Hole

- Discovered in 2014 [HM3, page 104] by Fiona H
  - A small hole, below Sunset Spot, not far off the main path (left when looking uphill) up to the Plateau
  - Short tube sloping downwards into small chamber (room to turn around kneeling), tube continued sloping downwards and got too tight (leg sized)
  - Not a good lead

### T2

- Mentioned in HM1, page 99, 1998
  - A tight strongly drafting rift passage that requires hammering

### The Freezer

- Mentioned in HM1, page 148, 2001
  - Further up the Gardener's World valley
  - One un-pushed lead at the end in a dodgy boulder choke
  - Contained a nice ice stalactite
- Spotted some coordinates on the [old Slovenia expo website](https://www.union.ic.ac.uk/rcc/caving/old/slovenia/report/cavedata)

### The Hairdryer

- Mentioned in HM1, page 99, 1998
  - "A slow shuffle into the bedding leads to a small, 8” hole. Beyond this restriction was black space, and the source of the very strong draft"
  - Noted in Jarv's requests for stories in HM2 Doesn't seem to have been investigated further

### TR1

2022:

<img src="images/TR1-2022.jpeg" width="25%"/>

- Discovered in 2014 [HM3, page 62]
  - A 1 x 1.5 m hole plugged with snow
  - Multiple entrances, some of which free climbable and reaches floor of ice and rubble
  - Left undescended
- Returned to in 2016 [HM3, page 188]
  - All leads were found to die quickly in rubble and ice

### TR2

2022:

<img src="images/TR2-2022.jpeg" width="44.5%"/>

- Discovered in 2014 [HM3, page 62]
  - Snow and rubble slope leads down to a low chamber with several blind avens
  - Terminal chamber reached after a short dug out crawl
- Revisited in 2022 by David and Rory
  - Near entrance just prior to crawl, on left space noticed under boulder, dug out, leading to chamber
  - Chamber terminates in too tight rift, Aven above chamber leads to undug light from surface
  - Draft disappears underneath climb down into chamber into rock floor
  - Thin person should investigate rift, might be possible to blast

### TR3

2022:

<img src="images/TR3-2022.jpeg" width="25%"/>

- Discovered in 2014 [HM3, page 62]
  - Entrance choke can be down-climbed to a small chamber with continuation requiring rock removal with tape, crowbar and chisel

### TR4

2022:

<img src="images/TR4-2022.jpeg" width="25%"/>

- Discovered in 2014 [HM3, page 63]
  - Obvious dig with upturned boulders near entrance

### TR5

2022:

<img src="images/TR5-2022.jpeg" width="25%"/>

- Discovered in 2014 [HM3, page 63]
  - Small free climb down
  - Draughting with easy digging

### TR6

2022:

<img src="images/TR6-2022.jpeg" width="25%"/>

- Discovered in 2014 [HM3, page 63]
  - Dig in vegetated shakehole
  - Closes down as boulders meet ceiling
  - Harder digging

### TR7

2022:

<img src="images/TR7-2022.jpeg" width="50%"/>

- Discovered in 2014 [HM3, page 63]
  - No obvious digging, both branches quickly become impassable

### TR8

2022:

<div>
  <img src="images/TR8-2022.jpeg" width="44.5%"/>
</div>

2023:

<div>
  <img src="images/TR8-2023-1.jpeg" width="44.5%"/>
  <img src="images/TR8-2023-2.jpeg" width="44.5%"/>
</div>

- Discovered in 2014 [HM3, page 63]
  - Ends in rift blocked by ice and cobbles
  - Would need serious digging if ice still present
- Revisited in 2023
  - New hole likely uncovered from ice plug melt revealed 10m continuation
  - Squeezing through leads to small chamber, second squeeze and then larger 5m wide, 10m long and 5m tall chamber
  - End chamber called Kevin's Wine Cellar, “I’d genuinely put stuff in there, like wine and cheese” -Kevin
  - Drafting quite a lot, and possible dig in the gravel with the draft coming from it. Mostly gravel so would need a lot of work

### TR9

2022:

<img src="images/TR9-2022.jpeg" width="25%"/>

- Discovered in 2014 [HM3, page 63]
  - Obvious way on is choked, no indication of how easy digging / rock removal might be
- Revisited in 2023
  - Howling draft coming out of the cave
  - Vocal connection to a shake hole behind the main entrance
  - A dig was found at the back in a tight crawl and pushed for 5m and continues on for about another 5m
  - Very strong draft coming from the crawl, which turns to the right into the mountain

### TR10

2023:

<div>
  <img src="images/TR10-2023-1.jpeg" width="44.5%"/>
  <img src="images/TR10-2023-2.jpeg" width="44.5%"/>
</div>

- Logged as TR10 (no idea when)
- Relogged 2023
  - Unclear whether this was ever descended but a 5m hole which looks enticing
  - Not descended due to lack of gear
  - A TR11 GPS point was also logged but couldn't find any cave there within a 15m radius

### U-Bend 571

- Another entrance to the main system that quickly connects with Primadona
- First mentioned in HM1, page 120, 2000
- Connected to Primadona in 2007 [HM2 draft, page 14]

### Vilinksa Jama

- Discovered in 2008 [HM2 draft, page 43]
  - Connects to Vrtnarija

### Vrtnarija, Gardener's World

- Entrance into the main system that began as a dig by Ben Ogborne [HM1, page 99, 1998]
- The entrance dig broke into the passage in 2000 [HM1, page 128]

'''